In [1]:
from django import setup
setup()

In [2]:
from django.utils import timezone
from entries.models import Entry, Nutrient

In [62]:
e1 = Entry(when=timezone.now(), what="something 4", quantity=2, category="c", measure="g")

In [4]:
e1.save()
e1.id

126

In [5]:
i1 = Nutrient()

In [6]:
i1.label = "sugar"
i1.unit = "g"
i1.category = "i"
i1.quantity = 4

In [3]:
from django.db.models import Sum, Count
Nutrient.objects.filter(category="o").aggregate(Sum("quantity"))

{'quantity__sum': 132.0}

## Reporting

In [4]:
from django.db.models.functions import TruncDay, TruncDate
from django.db.models import Sum, Count

In [5]:
import arrow
start_date = arrow.get("2016-11-16")
end_date = arrow.get("2016-12-06 23:59:59")
print(start_date, end_date)

2016-11-16T00:00:00+00:00 2016-12-06T23:59:59+00:00


In [ ]:
e1 = Entry(
    when=timezone.now(), 
    what="something 4", 
    quantity=2, 
    category=Entry.CATEGORIES.FOOD_CONSUMPTION, 
    measure="g")
e1.save()

e2 = Entry(
    when=timezone.now(), 
    what="something 4", 
    quantity=2, 
    category=Entry.CATEGORIES.PHYSICAL_ACTIVITY, 
    measure="g")

e2.save()

In [ ]:
e1n1 = Nutrient.objects.create(
    entry=e1,
    category=Nutrient.CATEGORIES.INTAKE,
    label="Energy",
    unit="kcal",
    quantity=140.5
)

e2n1 = Nutrient.objects.create(
    entry=e1,
    category=Nutrient.CATEGORIES.OUTTAKE,
    label="Energy",
    unit="kcal",
    quantity=10.5
)



In [6]:
Nutrient.get_energy_report(
    Nutrient.CATEGORIES.INTAKE, 
    timezone.make_aware(start_date.naive), 
    timezone.make_aware(arrow.utcnow().naive))

{datetime.date(2016, 11, 16): 0,
 datetime.date(2016, 11, 17): 0,
 datetime.date(2016, 11, 18): 0,
 datetime.date(2016, 11, 19): 0,
 datetime.date(2016, 11, 20): 0,
 datetime.date(2016, 11, 21): 0,
 datetime.date(2016, 11, 22): 0,
 datetime.date(2016, 11, 23): 0,
 datetime.date(2016, 11, 24): 0,
 datetime.date(2016, 11, 25): 0,
 datetime.date(2016, 11, 26): 0,
 datetime.date(2016, 11, 27): 0,
 datetime.date(2016, 11, 28): 0,
 datetime.date(2016, 11, 29): 0,
 datetime.date(2016, 11, 30): 90.0,
 datetime.date(2016, 12, 1): 28.0,
 datetime.date(2016, 12, 2): 0,
 datetime.date(2016, 12, 3): 0,
 datetime.date(2016, 12, 4): 0,
 datetime.date(2016, 12, 5): 0,
 datetime.date(2016, 12, 6): 0,
 datetime.date(2016, 12, 7): 0}

In [59]:
report = Nutrient.get_energy_report(
    Nutrient.CATEGORIES.OUTTAKE, 
    timezone.make_aware(start_date.naive), 
    timezone.make_aware(end_date.naive))

sorted(report.items())

[(datetime.date(2016, 11, 16), 0),
 (datetime.date(2016, 11, 17), 0),
 (datetime.date(2016, 11, 18), 0),
 (datetime.date(2016, 11, 19), 0),
 (datetime.date(2016, 11, 20), 0),
 (datetime.date(2016, 11, 21), 0),
 (datetime.date(2016, 11, 22), 0),
 (datetime.date(2016, 11, 23), 0),
 (datetime.date(2016, 11, 24), 0),
 (datetime.date(2016, 11, 25), 0),
 (datetime.date(2016, 11, 26), 0),
 (datetime.date(2016, 11, 27), 0),
 (datetime.date(2016, 11, 28), 0),
 (datetime.date(2016, 11, 29), 0),
 (datetime.date(2016, 11, 30), 0),
 (datetime.date(2016, 12, 1), 0),
 (datetime.date(2016, 12, 2), 1.32),
 (datetime.date(2016, 12, 3), 0),
 (datetime.date(2016, 12, 4), 0),
 (datetime.date(2016, 12, 5), 0),
 (datetime.date(2016, 12, 6), 12.5)]

In [38]:
a = (end_date - start_date)

In [52]:
records = Nutrient.objects.values("label", "unit").annotate(value=Sum("quantity"))
print(records.query.__str__())
for i in records:
    print(i)


SELECT "entries_nutrient"."label", "entries_nutrient"."unit", SUM("entries_nutrient"."quantity") AS "value" FROM "entries_nutrient" GROUP BY "entries_nutrient"."label", "entries_nutrient"."unit"
{'value': 13.82, 'unit': u'kcal', 'label': u'Energy'}
{'value': 8.0, 'unit': u'mg', 'label': u'fat'}


In [24]:
e2n1 = Nutrient.objects.create(
    entry=e1,
    category=Nutrient.CATEGORIES.OUTTAKE,
    label="fat",
    unit="mg",
    quantity=3
)


In [43]:
nutrients = Nutrient.objects.filter(
        category="o",
        entry__when__range=[timezone.make_aware(start_date.naive), 
            timezone.make_aware(end_date.naive)]
    ).values(
        "label", "unit"
    ).annotate(
        value=Sum("quantity")
    )

In [42]:
nutrients.query.__str__()

u'SELECT "entries_nutrient"."label", "entries_nutrient"."unit", SUM("entries_nutrient"."quantity") AS "value" FROM "entries_nutrient" INNER JOIN "entries_entry" ON ("entries_nutrient"."entry_id" = "entries_entry"."id") WHERE ("entries_nutrient"."category" = o AND "entries_entry"."when" BETWEEN 2016-11-16 00:00:00 AND 2016-12-06 23:59:59) GROUP BY "entries_nutrient"."label", "entries_nutrient"."unit"'

In [13]:
nutrients = Nutrient.get_nutrients_report(
    timezone.make_aware(start_date.naive), 
    timezone.make_aware(end_date.naive))

for i in nutrients:
    print(i)

{'value': 6.2, 'unit': u'mg', 'label': u'(+)-Catechin'}
{'value': 0.001, 'unit': u'g', 'label': u'10:0'}
{'value': 0.002, 'unit': u'g', 'label': u'12:0'}
{'value': 0.002, 'unit': u'g', 'label': u'14:0'}
{'value': 0.128, 'unit': u'g', 'label': u'16:0'}
{'value': 0.011, 'unit': u'g', 'label': u'16:1 undifferentiated'}
{'value': 0.013999999999999999, 'unit': u'g', 'label': u'18:0'}
{'value': 0.057999999999999996, 'unit': u'g', 'label': u'18:1 undifferentiated'}
{'value': 0.14200000000000002, 'unit': u'g', 'label': u'18:2 undifferentiated'}
{'value': 0.031, 'unit': u'g', 'label': u'18:3 undifferentiated'}
{'value': 0.082, 'unit': u'g', 'label': u'Alanine'}
{'value': 0.08499999999999999, 'unit': u'g', 'label': u'Arginine'}
{'value': 1.45, 'unit': u'g', 'label': u'Ash'}
{'value': 0.32899999999999996, 'unit': u'g', 'label': u'Aspartic acid'}
{'value': 0.1, 'unit': u'mg', 'label': u'Betaine'}
{'value': 21.0, 'unit': u'mg', 'label': u'Calcium, Ca'}
{'value': 29.34, 'unit': u'g', 'label': u'Carb

### Recommended Nutrients

In [23]:
import pandas as pd

In [46]:
pd.read_csv("/Users/muatik/Downloads/nutrients.csv", header=None)

,0,1,2,3,4,5
0,Total fat,g,65,Total lipid (fat),Proximates,g
1,Saturated fatty acids,g,20,"Fatty acids, total saturated",Lipids,g
2,Cholesterol,mg,300,Cholesterol,Proximates,mg
3,Sodium,mg,"2,400","Sodium, Na",Minerals,mg
4,Potassium,mg,"3,500","Potassium, K",Minerals,mg
5,Total carbohydrate,g,300,"Carbohydrate, by difference",Proximates,g
6,Fiber,g,25,"Fiber, total dietary",Proximates,g
7,Protein,g,50,Protein,Proximates,g
8,Vitamin A,IU,"5,000","Vitamin A, IU",Vitamins,IU
9,Vitamin C,mg,60,"Vitamin C, total ascorbic acid",Vitamins,mg


In [30]:
d = pd.read_csv("/Users/muatik/Downloads/nutrients.csv", header=None)
d = d.to_dict()

In [44]:
data = []
for i in range(len(d[0])):
    data.append({
        "label": d[0][i], 
        "unit": d[1][i], 
        "amount": d[2][i], 
        "usda_label": d[3][i], 
        "usda_category": d[4][i]})

In [42]:
data = {
    "headers": [
        "label",
        "unit",
        "amount",
        "usda_label",
        "usda_category"
    ],
    "labels": data
}

In [45]:
data

[{'amount': '65',
  'label': 'Total fat',
  'unit': 'g',
  'usda_category': 'Proximates',
  'usda_label': 'Total lipid (fat)'},
 {'amount': '20',
  'label': 'Saturated fatty acids',
  'unit': 'g',
  'usda_category': 'Lipids',
  'usda_label': 'Fatty acids, total saturated'},
 {'amount': '300',
  'label': 'Cholesterol',
  'unit': 'mg',
  'usda_category': 'Proximates',
  'usda_label': 'Cholesterol'},
 {'amount': '2,400',
  'label': 'Sodium',
  'unit': 'mg',
  'usda_category': 'Minerals',
  'usda_label': 'Sodium, Na'},
 {'amount': '3,500',
  'label': 'Potassium',
  'unit': 'mg',
  'usda_category': 'Minerals',
  'usda_label': 'Potassium, K'},
 {'amount': '300',
  'label': 'Total carbohydrate',
  'unit': 'g',
  'usda_category': 'Proximates',
  'usda_label': 'Carbohydrate, by difference'},
 {'amount': '25',
  'label': 'Fiber',
  'unit': 'g',
  'usda_category': 'Proximates',
  'usda_label': 'Fiber, total dietary'},
 {'amount': '50',
  'label': 'Protein',
  'unit': 'g',
  'usda_category': 'Prox

## User management

In [5]:
from django.contrib.auth.models import User
from entries.models import UserProfile

In [32]:
u = User(email="muatik@gmail.com", first_name="mustafa")
u.save()


In [58]:
u.userprofile = UserProfile(height=176, birth_date=timezone.make_aware(arrow.utcnow().naive))
u.userprofile.gender = UserProfile.MALE
u.userprofile.save()

In [59]:
u.last_name = "atik"
u.save()

In [6]:
u = User.objects.all()[0]

In [8]:
q, d = Nutrient.get_energy_report(u, "i", 
                        timezone.make_aware(start_date.naive), 
                        timezone.make_aware(end_date.naive))

In [9]:
q.query.__str__()

u'SELECT "entries_entry"."when", django_datetime_cast_date("entries_entry"."when", UTC) AS "date", SUM("entries_nutrient"."quantity") AS "value" FROM "entries_nutrient" INNER JOIN "entries_entry" ON ("entries_nutrient"."entry_id" = "entries_entry"."id") WHERE ("entries_nutrient"."category" = i AND "entries_entry"."user_id" = 1 AND "entries_entry"."when" BETWEEN 2016-11-16 00:00:00 AND 2016-12-06 23:59:59 AND "entries_nutrient"."unit" = kcal AND "entries_nutrient"."label" = Energy) GROUP BY django_datetime_cast_date("entries_entry"."when", UTC), "entries_entry"."when"'

In [23]:
Nutrient.objects.filter(
            entry=Entry.objects.get(id=33),
            label="Energy",
            unit="kcal",
            category=Nutrient.CATEGORIES.OUTTAKE
        ).aggregate(Sum("quantity"))

{'quantity__sum': None}

In [24]:
data = [
    {
        "id": 1,
        "code": "01003",
        "METS": 14,
        "activityGroupId": 1,
        "description": "bicycling, mountain, uphill, vigorous"
    },
    {
        "id": 2,
        "code": "01004",
        "METS": 16,
        "activityGroupId": 1,
        "description": "bicycling, mountain, competitive, racing"
    },
    {
        "id": 3,
        "code": "01008",
        "METS": 8.5,
        "activityGroupId": 1,
        "description": "bicycling, BMX"
    },
    {
        "id": 4,
        "code": "01009",
        "METS": 8.5,
        "activityGroupId": 1,
        "description": "bicycling, mountain, general"
    },
    {
        "id": 5,
        "code": "01010",
        "METS": 4,
        "activityGroupId": 1,
        "description": "bicycling, <10 mph, leisure, to work or for pleasure "
                       "(Taylor code 115)"
    },
    {
        "id": 6,
        "code": "01011",
        "METS": 6.8,
        "activityGroupId": 1,
        "description": "bicycling, to/from work, self selected pace"
    },
    {
        "id": 7,
        "code": "01013",
        "METS": 5.8,
        "activityGroupId": 1,
        "description": "bicycling, on dirt or farm road, moderate pace"
    },
    {
        "id": 8,
        "code": "01015",
        "METS": 7.5,
        "activityGroupId": 1,
        "description": "bicycling, general"
    },
    {
        "id": 9,
        "code": "01018",
        "METS": 3.5,
        "activityGroupId": 1,
        "description": "bicycling, leisure, 5.5 mph"
    },
    {
        "id": 10,
        "code": "01019",
        "METS": 5.8,
        "activityGroupId": 1,
        "description": "bicycling, leisure, 9.4 mph"
    },
    {
        "id": 11,
        "code": "01020",
        "METS": 6.8,
        "activityGroupId": 1,
        "description": "bicycling, 10-11.9 mph, leisure, slow, light effort"
    },
    {
        "id": 12,
        "code": "01030",
        "METS": 8,
        "activityGroupId": 1,
        "description": "bicycling, 12-13.9 mph, leisure, moderate effort"
    },
    {
        "id": 13,
        "code": "01040",
        "METS": 10,
        "activityGroupId": 1,
        "description": "bicycling, 14-15.9 mph, racing or leisure, fast, "
                       "vigorous effort"
    },
    {
        "id": 14,
        "code": "01050",
        "METS": 12,
        "activityGroupId": 1,
        "description": "bicycling, 16-19 mph, racing/not drafting or > 19 mph "
                       "drafting, very fast, racing general"
    },
    {
        "id": 15,
        "code": "01060",
        "METS": 15.8,
        "activityGroupId": 1,
        "description": "bicycling, > 20 mph, racing, not drafting"
    },
    {
        "id": 16,
        "code": "01065",
        "METS": 8.5,
        "activityGroupId": 1,
        "description": "bicycling, 12 mph, seated, hands on brake hoods or "
                       "bar drops, 80 rpm"
    },
    {
        "id": 17,
        "code": "01066",
        "METS": 9,
        "activityGroupId": 1,
        "description": "bicycling, 12 mph, standing, hands on brake hoods, "
                       "60 rpm"
    },
    {
        "id": 18,
        "code": "01070",
        "METS": 5,
        "activityGroupId": 1,
        "description": "unicycling"
    },
    {
        "id": 19,
        "code": "02001",
        "METS": 2.3,
        "activityGroupId": 2,
        "description": "activity promoting video game (e.g., Wii Fit), "
                       "light effort (e.g., balance, yoga)"
    },
    {
        "id": 20,
        "code": "02003",
        "METS": 3.8,
        "activityGroupId": 2,
        "description": "activity promoting video game (e.g., Wii Fit), "
                       "moderate effort (e.g., aerobic, resistance)"
    },
    {
        "id": 21,
        "code": "02005",
        "METS": 7.2,
        "activityGroupId": 2,
        "description": "activity promoting video/arcade game (e.g., "
                       "Exergaming, Dance Dance Revolution), vigorous effort"
    },
    {
        "id": 22,
        "code": "02008",
        "METS": 5,
        "activityGroupId": 2,
        "description": "army type obstacle course exercise, boot camp "
                       "training program"
    },
    {
        "id": 23,
        "code": "02010",
        "METS": 7,
        "activityGroupId": 2,
        "description": "bicycling, stationary, general"
    },
    {
        "id": 24,
        "code": "02011",
        "METS": 3.5,
        "activityGroupId": 2,
        "description": "bicycling, stationary, 30-50 watts, very light to "
                       "light effort"
    },
    {
        "id": 25,
        "code": "02012",
        "METS": 6.8,
        "activityGroupId": 2,
        "description": "bicycling, stationary, 90-100 watts, moderate to "
                       "vigorous effort"
    },
    {
        "id": 26,
        "code": "02013",
        "METS": 8.8,
        "activityGroupId": 2,
        "description": "bicycling, stationary, 101-160 watts, vigorous effort"
    },
    {
        "id": 27,
        "code": "02014",
        "METS": 11,
        "activityGroupId": 2,
        "description": "bicycling, stationary, 161-200 watts, vigorous effort"
    },
    {
        "id": 28,
        "code": "02015",
        "METS": 14,
        "activityGroupId": 2,
        "description": "bicycling, stationary, 201-270 watts, very vigorous "
                       "effort"
    },
    {
        "id": 29,
        "code": "02017",
        "METS": 4.8,
        "activityGroupId": 2,
        "description": "bicycling, stationary, 51-89 watts, light-to-moderate"
                       " effort"
    },
    {
        "id": 30,
        "code": "02019",
        "METS": 8.5,
        "activityGroupId": 2,
        "description": "bicycling, stationary, RPM/Spin bike class"
    },
    {
        "id": 31,
        "code": "02020",
        "METS": 8,
        "activityGroupId": 2,
        "description": "calisthenics (e.g., push ups, sit ups, pull-ups, "
                       "jumping jacks), vigorous effort"
    },
    {
        "id": 32,
        "code": "02022",
        "METS": 3.8,
        "activityGroupId": 2,
        "description": "calisthenics (e.g., push ups, sit ups, pull-ups, "
                       "lunges), moderate effort"
    },
    {
        "id": 33,
        "code": "02024",
        "METS": 2.8,
        "activityGroupId": 2,
        "description": "calisthenics (e.g., situps, abdominal crunches), "
                       "light effort"
    },
    {
        "id": 34,
        "code": "02030",
        "METS": 3.5,
        "activityGroupId": 2,
        "description": "calisthenics, light or moderate effort, general ("
                       "e.g., back exercises), going up & down from floor ("
                       "Taylor code 150)"
    },
    {
        "id": 35,
        "code": "02035",
        "METS": 4.3,
        "activityGroupId": 2,
        "description": "circuit training, moderate effort"
    },
    {
        "id": 36,
        "code": "02040",
        "METS": 8,
        "activityGroupId": 2,
        "description": "circuit training, including kettlebells, some aerobic "
                       "movement with minimal rest, general, vigorous intensity"
    },
    {
        "id": 37,
        "code": "02045",
        "METS": 3.5,
        "activityGroupId": 2,
        "description": "CurvesTM exercise routines in women"
    },
    {
        "id": 38,
        "code": "02048",
        "METS": 5,
        "activityGroupId": 2,
        "description": "Elliptical trainer, moderate effort"
    },
    {
        "id": 39,
        "code": "02050",
        "METS": 6,
        "activityGroupId": 2,
        "description": "resistance training (weight lifting, free weight, "
                       "nautilus or universal), power lifting or body "
                       "building, vigorous effort (Taylor code 210)"
    },
    {
        "id": 40,
        "code": "02052",
        "METS": 5,
        "activityGroupId": 2,
        "description": "resistance (weight) training, squats , slow or "
                       "explosive effort"
    },
    {
        "id": 41,
        "code": "02054",
        "METS": 3.5,
        "activityGroupId": 2,
        "description": "resistance (weight) training, multiple exercises, "
                       "8-15 repetitions at varied resistance"
    },
    {
        "id": 42,
        "code": "02060",
        "METS": 5.5,
        "activityGroupId": 2,
        "description": "health club exercise, general (Taylor code 160)"
    },
    {
        "id": 43,
        "code": "02061",
        "METS": 5,
        "activityGroupId": 2,
        "description": "health club exercise classes, general, gym/weight "
                       "training combined in one visit"
    },
    {
        "id": 44,
        "code": "02062",
        "METS": 7.8,
        "activityGroupId": 2,
        "description": "health club exercise, conditioning classes"
    },
    {
        "id": 45,
        "code": "02064",
        "METS": 3.8,
        "activityGroupId": 2,
        "description": "home exercise, general"
    },
    {
        "id": 46,
        "code": "02065",
        "METS": 9,
        "activityGroupId": 2,
        "description": "stair-treadmill ergometer, general"
    },
    {
        "id": 47,
        "code": "02068",
        "METS": 12.3,
        "activityGroupId": 2,
        "description": "rope skipping, general"
    },
    {
        "id": 48,
        "code": "02070",
        "METS": 6,
        "activityGroupId": 2,
        "description": "rowing, stationary ergometer, general, vigorous effort"
    },
    {
        "id": 49,
        "code": "02071",
        "METS": 4.8,
        "activityGroupId": 2,
        "description": "rowing, stationary, general, moderate effort"
    },
    {
        "id": 50,
        "code": "02072",
        "METS": 7,
        "activityGroupId": 2,
        "description": "rowing, stationary, 100 watts, moderate effort"
    },
    {
        "id": 51,
        "code": "02073",
        "METS": 8.5,
        "activityGroupId": 2,
        "description": "rowing, stationary, 150 watts, vigorous effort"
    },
    {
        "id": 52,
        "code": "02074",
        "METS": 12,
        "activityGroupId": 2,
        "description": "rowing, stationary, 200 watts, very vigorous effort"
    },
    {
        "id": 53,
        "code": "02080",
        "METS": 6.8,
        "activityGroupId": 2,
        "description": "ski machine, general"
    },
    {
        "id": 54,
        "code": "02085",
        "METS": 11,
        "activityGroupId": 2,
        "description": "slide board exercise, general"
    },
    {
        "id": 55,
        "code": "02090",
        "METS": 6,
        "activityGroupId": 2,
        "description": "slimnastics, jazzercise"
    },
    {
        "id": 56,
        "code": "02101",
        "METS": 2.3,
        "activityGroupId": 2,
        "description": "stretching, mild"
    },
    {
        "id": 57,
        "code": "02105",
        "METS": 3,
        "activityGroupId": 2,
        "description": "pilates, general"
    },
    {
        "id": 58,
        "code": "02110",
        "METS": 6.8,
        "activityGroupId": 2,
        "description": "teaching exercise class (e.g., aerobic, water)"
    },
    {
        "id": 59,
        "code": "02112",
        "METS": 2.8,
        "activityGroupId": 2,
        "description": "therapeutic exercise ball, Fitball exercise"
    },
    {
        "id": 60,
        "code": "02115",
        "METS": 2.8,
        "activityGroupId": 2,
        "description": "upper body exercise, arm ergometer"
    },
    {
        "id": 61,
        "code": "02117",
        "METS": 4.3,
        "activityGroupId": 2,
        "description": "upper body exercise, stationary bicycle - Airdyne ("
                       "arms only) 40 rpm, moderate"
    },
    {
        "id": 62,
        "code": "02120",
        "METS": 5.3,
        "activityGroupId": 2,
        "description": "water aerobics, water calisthenics, water exercise"
    },
    {
        "id": 63,
        "code": "02135",
        "METS": 1.3,
        "activityGroupId": 2,
        "description": "whirlpool, sitting"
    },
    {
        "id": 64,
        "code": "02140",
        "METS": 2.3,
        "activityGroupId": 2,
        "description": "video exercise workouts, TV conditioning programs ("
                       "e.g., yoga, stretching), light effort"
    },
    {
        "id": 65,
        "code": "02143",
        "METS": 4,
        "activityGroupId": 2,
        "description": "video exercise workouts, TV conditioning programs ("
                       "e.g., cardio-resistance), moderate effort"
    },
    {
        "id": 66,
        "code": "02146",
        "METS": 6,
        "activityGroupId": 2,
        "description": "video exercise workouts, TV conditioning programs ("
                       "e.g., cardio-resistance), vigorous effort"
    },
    {
        "id": 67,
        "code": "02150",
        "METS": 2.5,
        "activityGroupId": 2,
        "description": "yoga, Hatha"
    },
    {
        "id": 68,
        "code": "02160",
        "METS": 4,
        "activityGroupId": 2,
        "description": "yoga, Power"
    },
    {
        "id": 69,
        "code": "02170",
        "METS": 2,
        "activityGroupId": 2,
        "description": "yoga, Nadisodhana"
    },
    {
        "id": 70,
        "code": "02180",
        "METS": 3.3,
        "activityGroupId": 2,
        "description": "yoga, Surya Namaskar"
    },
    {
        "id": 71,
        "code": "02200",
        "METS": 5.3,
        "activityGroupId": 2,
        "description": "native New Zealander physical activities (e.g., "
                       "Haka Powhiri, Moteatea, Waita Tira, Whakawatea, "
                       "etc.), general, moderate effort"
    },
    {
        "id": 72,
        "code": "02205",
        "METS": 6.8,
        "activityGroupId": 2,
        "description": "native New Zealander physical activities (e.g., Haka, "
                       "Taiahab), general, vigorous effort"
    },
    {
        "id": 73,
        "code": "03010",
        "METS": 5,
        "activityGroupId": 3,
        "description": "ballet, modern, or jazz, general, rehearsal or class"
    },
    {
        "id": 74,
        "code": "03012",
        "METS": 6.8,
        "activityGroupId": 3,
        "description": "ballet, modern, or jazz, performance, vigorous effort"
    },
    {
        "id": 75,
        "code": "03014",
        "METS": 4.8,
        "activityGroupId": 3,
        "description": "tap"
    },
    {
        "id": 76,
        "code": "03015",
        "METS": 7.3,
        "activityGroupId": 3,
        "description": "aerobic, general"
    },
    {
        "id": 77,
        "code": "03016",
        "METS": 7.5,
        "activityGroupId": 3,
        "description": "aerobic, step, with 6 - 8 inch step"
    },
    {
        "id": 78,
        "code": "03017",
        "METS": 9.5,
        "activityGroupId": 3,
        "description": "aerobic, step, with 10 - 12 inch step"
    },
    {
        "id": 79,
        "code": "03018",
        "METS": 5.5,
        "activityGroupId": 3,
        "description": "aerobic, step, with 4-inch step"
    },
    {
        "id": 80,
        "code": "03019",
        "METS": 8.5,
        "activityGroupId": 3,
        "description": "bench step class, general"
    },
    {
        "id": 81,
        "code": "03020",
        "METS": 5,
        "activityGroupId": 3,
        "description": "aerobic, low impact"
    },
    {
        "id": 82,
        "code": "03021",
        "METS": 7.3,
        "activityGroupId": 3,
        "description": "aerobic, high impact"
    },
    {
        "id": 83,
        "code": "03022",
        "METS": 10,
        "activityGroupId": 3,
        "description": "aerobic dance wearing 10-15 lb weights"
    },
    {
        "id": 84,
        "code": "03025",
        "METS": 4.5,
        "activityGroupId": 3,
        "description": "ethnic or cultural dancing (e.g., Greek, Middle "
                       "Eastern, hula, salsa, merengue, bamba y plena, "
                       "flamenco, belly, and swing)"
    },
    {
        "id": 85,
        "code": "03030",
        "METS": 5.5,
        "activityGroupId": 3,
        "description": "ballroom, fast (Taylor code 125)"
    },
    {
        "id": 86,
        "code": "03031",
        "METS": 7.8,
        "activityGroupId": 3,
        "description": "general dancing (e.g., disco, folk, Irish step "
                       "dancing, line dancing, polka, contra, country)"
    },
    {
        "id": 87,
        "code": "03038",
        "METS": 11.3,
        "activityGroupId": 3,
        "description": "ballroom dancing, competitive, general"
    },
    {
        "id": 88,
        "code": "03040",
        "METS": 3,
        "activityGroupId": 3,
        "description": "ballroom, slow (e.g., waltz, foxtrot, slow dancing, "
                       "samba, tango, 19th century dance, mambo, cha cha)"
    },
    {
        "id": 89,
        "code": "03050",
        "METS": 5.5,
        "activityGroupId": 3,
        "description": "Anishinaabe Jingle Dancing"
    },
    {
        "id": 90,
        "code": "03060",
        "METS": 3.5,
        "activityGroupId": 3,
        "description": "Caribbean dance (Abakua, Beguine, Bellair, Bongo, "
                       "Brukins, Caribbean Quadrills, Dinki Mini, Gere, "
                       "Gumbay, Ibo, Jonkonnu, Kumina, Oreisha, Jambu)"
    },
    {
        "id": 91,
        "code": "04001",
        "METS": 3.5,
        "activityGroupId": 4,
        "description": "fishing, general"
    },
    {
        "id": 92,
        "code": "04005",
        "METS": 4.5,
        "activityGroupId": 4,
        "description": "fishing, crab fishing"
    },
    {
        "id": 93,
        "code": "04007",
        "METS": 4,
        "activityGroupId": 4,
        "description": "fishing, catching fish with hands"
    },
    {
        "id": 94,
        "code": "04010",
        "METS": 4.3,
        "activityGroupId": 4,
        "description": "fishing related, digging worms, with shovel"
    },
    {
        "id": 95,
        "code": "04020",
        "METS": 4,
        "activityGroupId": 4,
        "description": "fishing from river bank and walking"
    },
    {
        "id": 96,
        "code": "04030",
        "METS": 2,
        "activityGroupId": 4,
        "description": "fishing from boat or canoe, sitting"
    },
    {
        "id": 97,
        "code": "04040",
        "METS": 3.5,
        "activityGroupId": 4,
        "description": "fishing from river bank, standing (Taylor code 660)"
    },
    {
        "id": 98,
        "code": "04050",
        "METS": 6,
        "activityGroupId": 4,
        "description": "fishing in stream, in waders (Taylor code 670)"
    },
    {
        "id": 99,
        "code": "04060",
        "METS": 2,
        "activityGroupId": 4,
        "description": "fishing, ice, sitting"
    },
    {
        "id": 100,
        "code": "04061",
        "METS": 1.8,
        "activityGroupId": 4,
        "description": "fishing, jog or line, standing, general"
    },
    {
        "id": 101,
        "code": "04062",
        "METS": 3.5,
        "activityGroupId": 4,
        "description": "fishing, dip net, setting net and retrieving fish, "
                       "general"
    },
    {
        "id": 102,
        "code": "04063",
        "METS": 3.8,
        "activityGroupId": 4,
        "description": "fishing, set net, setting net and retrieving fish, "
                       "general"
    },
    {
        "id": 103,
        "code": "04064",
        "METS": 3,
        "activityGroupId": 4,
        "description": "fishing, fishing wheel, setting net and retrieving "
                       "fish, general"
    },
    {
        "id": 104,
        "code": "04065",
        "METS": 2.3,
        "activityGroupId": 4,
        "description": "fishing with a spear, standing"
    },
    {
        "id": 105,
        "code": "04070",
        "METS": 2.5,
        "activityGroupId": 4,
        "description": "hunting, bow and arrow, or crossbow"
    },
    {
        "id": 106,
        "code": "04080",
        "METS": 6,
        "activityGroupId": 4,
        "description": "hunting, deer, elk, large game (Taylor code 170)"
    },
    {
        "id": 107,
        "code": "04081",
        "METS": 11.3,
        "activityGroupId": 4,
        "description": "hunting large game, dragging carcass"
    },
    {
        "id": 108,
        "code": "04083",
        "METS": 4,
        "activityGroupId": 4,
        "description": "hunting large marine animals"
    },
    {
        "id": 109,
        "code": "04085",
        "METS": 2.5,
        "activityGroupId": 4,
        "description": "hunting large game, from a hunting stand, limited "
                       "walking"
    },
    {
        "id": 110,
        "code": "04086",
        "METS": 2,
        "activityGroupId": 4,
        "description": "hunting large game from a car, plane, or boat"
    },
    {
        "id": 111,
        "code": "04090",
        "METS": 2.5,
        "activityGroupId": 4,
        "description": "hunting, duck, wading"
    },
    {
        "id": 112,
        "code": "04095",
        "METS": 3,
        "activityGroupId": 4,
        "description": "hunting, flying fox, squirrel"
    },
    {
        "id": 113,
        "code": "04100",
        "METS": 5,
        "activityGroupId": 4,
        "description": "hunting, general"
    },
    {
        "id": 114,
        "code": "04110",
        "METS": 6,
        "activityGroupId": 4,
        "description": "hunting, pheasants or grouse (Taylor code 680)"
    },
    {
        "id": 115,
        "code": "04115",
        "METS": 3.3,
        "activityGroupId": 4,
        "description": "hunting, birds"
    },
    {
        "id": 116,
        "code": "04120",
        "METS": 5,
        "activityGroupId": 4,
        "description": "hunting, rabbit, squirrel, prairie chick, raccoon, "
                       "small game (Taylor code 690)"
    },
    {
        "id": 117,
        "code": "04123",
        "METS": 3.3,
        "activityGroupId": 4,
        "description": "hunting, pigs, wild"
    },
    {
        "id": 118,
        "code": "04124",
        "METS": 2,
        "activityGroupId": 4,
        "description": "trapping game, general"
    },
    {
        "id": 119,
        "code": "04125",
        "METS": 9.5,
        "activityGroupId": 4,
        "description": "hunting, hiking with hunting gear"
    },
    {
        "id": 120,
        "code": "04130",
        "METS": 2.5,
        "activityGroupId": 4,
        "description": "pistol shooting or trap shooting, standing"
    },
    {
        "id": 121,
        "code": "04140",
        "METS": 2.3,
        "activityGroupId": 4,
        "description": "rifle exercises, shooting, lying down"
    },
    {
        "id": 122,
        "code": "04145",
        "METS": 2.5,
        "activityGroupId": 4,
        "description": "rifle exercises, shooting, kneeling or standing"
    },
    {
        "id": 123,
        "code": "05010",
        "METS": 3.3,
        "activityGroupId": 5,
        "description": "cleaning, sweeping carpet or floors, general"
    },
    {
        "id": 124,
        "code": "05011",
        "METS": 2.3,
        "activityGroupId": 5,
        "description": "cleaning, sweeping, slow, light effort"
    },
    {
        "id": 125,
        "code": "05012",
        "METS": 3.8,
        "activityGroupId": 5,
        "description": "cleaning, sweeping, slow, moderate effort"
    },
    {
        "id": 126,
        "code": "05020",
        "METS": 3.5,
        "activityGroupId": 5,
        "description": "cleaning, heavy or major (e.g. wash car, "
                       "wash windows, clean garage), moderate effort"
    },
    {
        "id": 127,
        "code": "05021",
        "METS": 3.5,
        "activityGroupId": 5,
        "description": "cleaning, mopping, standing, moderate effort"
    },
    {
        "id": 128,
        "code": "05022",
        "METS": 3.2,
        "activityGroupId": 5,
        "description": "cleaning windows, washing windows, general"
    },
    {
        "id": 129,
        "code": "05023",
        "METS": 2.5,
        "activityGroupId": 5,
        "description": "mopping, standing, light effort"
    },
    {
        "id": 130,
        "code": "05024",
        "METS": 4.5,
        "activityGroupId": 5,
        "description": "polishing floors, standing, walking slowly, "
                       "using electric polishing machine"
    },
    {
        "id": 131,
        "code": "05025",
        "METS": 2.8,
        "activityGroupId": 5,
        "description": "multiple household tasks all at once, light effort"
    },
    {
        "id": 132,
        "code": "05026",
        "METS": 3.5,
        "activityGroupId": 5,
        "description": "multiple household tasks all at once, moderate effort"
    },
    {
        "id": 133,
        "code": "05027",
        "METS": 4.3,
        "activityGroupId": 5,
        "description": "multiple household tasks all at once, vigorous effort"
    },
    {
        "id": 134,
        "code": "05030",
        "METS": 3.3,
        "activityGroupId": 5,
        "description": "cleaning, house or cabin, general, moderate effort"
    },
    {
        "id": 135,
        "code": "05032",
        "METS": 2.3,
        "activityGroupId": 5,
        "description": "dusting or polishing furniture, general"
    },
    {
        "id": 136,
        "code": "05035",
        "METS": 3.3,
        "activityGroupId": 5,
        "description": "kitchen activity, general, (e.g., cooking, washing "
                       "dishes, cleaning up), moderate effort"
    },
    {
        "id": 137,
        "code": "05040",
        "METS": 2.5,
        "activityGroupId": 5,
        "description": "cleaning, general (straightening up, changing linen, "
                       "carrying out trash, light effort"
    },
    {
        "id": 138,
        "code": "05041",
        "METS": 1.8,
        "activityGroupId": 5,
        "description": "wash dishes, standing or in general (not broken into "
                       "stand/walk components)"
    },
    {
        "id": 139,
        "code": "05042",
        "METS": 2.5,
        "activityGroupId": 5,
        "description": "wash dishes, clearing dishes from table, walking, "
                       "light effort"
    },
    {
        "id": 140,
        "code": "05043",
        "METS": 3.3,
        "activityGroupId": 5,
        "description": "vacuuming, general, moderate effort"
    },
    {
        "id": 141,
        "code": "05044",
        "METS": 3,
        "activityGroupId": 5,
        "description": "butchering animals, small"
    },
    {
        "id": 142,
        "code": "05045",
        "METS": 6,
        "activityGroupId": 5,
        "description": "butchering animal, large, vigorous effort"
    },
    {
        "id": 143,
        "code": "05046",
        "METS": 2.3,
        "activityGroupId": 5,
        "description": "cutting and smoking fish, drying fish or meat"
    },
    {
        "id": 144,
        "code": "05048",
        "METS": 4,
        "activityGroupId": 5,
        "description": "tanning hides, general"
    },
    {
        "id": 145,
        "code": "05049",
        "METS": 3.5,
        "activityGroupId": 5,
        "description": "cooking or food preparation, moderate effort"
    },
    {
        "id": 146,
        "code": "05050",
        "METS": 2,
        "activityGroupId": 5,
        "description": "cooking or food preparation - standing or sitting or "
                       "in general (not broken into stand/walk components), "
                       "manual appliances, light effort"
    },
    {
        "id": 147,
        "code": "05051",
        "METS": 2.5,
        "activityGroupId": 5,
        "description": "serving food, setting table, implied walking or "
                       "standing"
    },
    {
        "id": 148,
        "code": "05052",
        "METS": 2.5,
        "activityGroupId": 5,
        "description": "cooking or food preparation, walking"
    },
    {
        "id": 149,
        "code": "05053",
        "METS": 2.5,
        "activityGroupId": 5,
        "description": "feeding household animals"
    },
    {
        "id": 150,
        "code": "05055",
        "METS": 2.5,
        "activityGroupId": 5,
        "description": "putting away groceries (e.g. carrying groceries, "
                       "shopping without a grocery cart), carrying packages"
    },
    {
        "id": 151,
        "code": "05056",
        "METS": 7.5,
        "activityGroupId": 5,
        "description": "carrying groceries upstairs"
    },
    {
        "id": 152,
        "code": "05057",
        "METS": 3,
        "activityGroupId": 5,
        "description": "cooking Indian bread on an outside stove"
    },
    {
        "id": 153,
        "code": "05060",
        "METS": 2.3,
        "activityGroupId": 5,
        "description": "food shopping with or without a grocery cart, "
                       "standing or walking"
    },
    {
        "id": 154,
        "code": "05065",
        "METS": 2.3,
        "activityGroupId": 5,
        "description": "non-food shopping, with or without a cart, standing "
                       "or walking"
    },
    {
        "id": 155,
        "code": "05070",
        "METS": 1.8,
        "activityGroupId": 5,
        "description": "ironing"
    },
    {
        "id": 156,
        "code": "05080",
        "METS": 1.3,
        "activityGroupId": 5,
        "description": "knitting, sewing, light effort, wrapping presents, "
                       "sitting"
    },
    {
        "id": 157,
        "code": "05082",
        "METS": 2.8,
        "activityGroupId": 5,
        "description": "sewing with a machine"
    },
    {
        "id": 158,
        "code": "05090",
        "METS": 2,
        "activityGroupId": 5,
        "description": "laundry, fold or hang clothes, put clothes in washer "
                       "or dryer, packing suitcase, washing clothes by hand, "
                       "implied standing, light effort"
    },
    {
        "id": 159,
        "code": "05092",
        "METS": 4,
        "activityGroupId": 5,
        "description": "laundry, hanging wash, washing clothes by hand, "
                       "moderate effort"
    },
    {
        "id": 160,
        "code": "05095",
        "METS": 2.3,
        "activityGroupId": 5,
        "description": "laundry, putting away clothes, gathering clothes to "
                       "pack, putting away laundry, implied walking"
    },
    {
        "id": 161,
        "code": "05100",
        "METS": 3.3,
        "activityGroupId": 5,
        "description": "making bed, changing linens"
    },
    {
        "id": 162,
        "code": "05110",
        "METS": 5,
        "activityGroupId": 5,
        "description": "maple syruping/sugar bushing (including carrying "
                       "buckets, carrying wood)"
    },
    {
        "id": 163,
        "code": "05120",
        "METS": 5.8,
        "activityGroupId": 5,
        "description": "moving furniture, household items, carrying boxes"
    },
    {
        "id": 164,
        "code": "05121",
        "METS": 5,
        "activityGroupId": 5,
        "description": "moving, lifting light loads"
    },
    {
        "id": 165,
        "code": "05125",
        "METS": 4.8,
        "activityGroupId": 5,
        "description": "organizing room"
    },
    {
        "id": 166,
        "code": "05130",
        "METS": 3.5,
        "activityGroupId": 5,
        "description": "scrubbing floors, on hands and knees, scrubbing "
                       "bathroom, bathtub, moderate effort"
    },
    {
        "id": 167,
        "code": "05131",
        "METS": 2,
        "activityGroupId": 5,
        "description": "scrubbing floors, on hands and knees, scrubbing "
                       "bathroom, bathtub, light effort"
    },
    {
        "id": 168,
        "code": "05132",
        "METS": 6.5,
        "activityGroupId": 5,
        "description": "scrubbing floors, on hands and knees, scrubbing "
                       "bathroom, bathtub, vigorous effort"
    },
    {
        "id": 169,
        "code": "05140",
        "METS": 4,
        "activityGroupId": 5,
        "description": "sweeping garage, sidewalk or outside of house"
    },
    {
        "id": 170,
        "code": "05146",
        "METS": 3.5,
        "activityGroupId": 5,
        "description": "standing, packing/unpacking boxes, occasional lifting "
                       "of lightweight household items, loading or unloading "
                       "items in car, moderate effort"
    },
    {
        "id": 171,
        "code": "05147",
        "METS": 3,
        "activityGroupId": 5,
        "description": "implied walking, putting away household items, "
                       "moderate effort"
    },
    {
        "id": 172,
        "code": "05148",
        "METS": 2.5,
        "activityGroupId": 5,
        "description": "watering plants"
    },
    {
        "id": 173,
        "code": "05149",
        "METS": 2.5,
        "activityGroupId": 5,
        "description": "building a fire inside"
    },
    {
        "id": 174,
        "code": "05150",
        "METS": 9,
        "activityGroupId": 5,
        "description": "moving household items upstairs, carrying boxes or "
                       "furniture"
    },
    {
        "id": 175,
        "code": "05160",
        "METS": 2,
        "activityGroupId": 5,
        "description": "standing, light effort tasks (pump gas, change light "
                       "bulb, etc.)"
    },
    {
        "id": 176,
        "code": "05165",
        "METS": 3.5,
        "activityGroupId": 5,
        "description": "walking, moderate effort tasks, non-cleaning ("
                       "readying to leave, shut/lock doors, close windows, "
                       "etc.)"
    },
    {
        "id": 177,
        "code": "05170",
        "METS": 2.2,
        "activityGroupId": 5,
        "description": "sitting, playing with child(ren), light effort, "
                       "only active periods"
    },
    {
        "id": 178,
        "code": "05171",
        "METS": 2.8,
        "activityGroupId": 5,
        "description": "standing, playing with child(ren) light effort, "
                       "only active periods"
    },
    {
        "id": 179,
        "code": "05175",
        "METS": 3.5,
        "activityGroupId": 5,
        "description": "walking/running, playing with child(ren), moderate "
                       "effort, only active periods"
    },
    {
        "id": 180,
        "code": "05180",
        "METS": 5.8,
        "activityGroupId": 5,
        "description": "walking/running, playing with child(ren), vigorous "
                       "effort, only active periods"
    },
    {
        "id": 181,
        "code": "05181",
        "METS": 3,
        "activityGroupId": 5,
        "description": "walking and carrying small child, child weighing 15 "
                       "lbs or more"
    },
    {
        "id": 182,
        "code": "05182",
        "METS": 2.3,
        "activityGroupId": 5,
        "description": "walking and carrying small child, child weighing less"
                       " than 15 lbs"
    },
    {
        "id": 183,
        "code": "05183",
        "METS": 2,
        "activityGroupId": 5,
        "description": "standing, holding child"
    },
    {
        "id": 184,
        "code": "05184",
        "METS": 2.5,
        "activityGroupId": 5,
        "description": "child care, infant, general"
    },
    {
        "id": 185,
        "code": "05185",
        "METS": 2,
        "activityGroupId": 5,
        "description": "child care, sitting/kneeling (e.g., dressing, "
                       "bathing, grooming, feeding, occasional lifting of "
                       "child), light effort, general"
    },
    {
        "id": 186,
        "code": "05186",
        "METS": 3,
        "activityGroupId": 5,
        "description": "child care, standing (e.g., dressing, bathing, "
                       "grooming, feeding, occasional lifting of child), "
                       "moderate effort"
    },
    {
        "id": 187,
        "code": "05188",
        "METS": 1.5,
        "activityGroupId": 5,
        "description": "reclining with baby"
    },
    {
        "id": 188,
        "code": "05189",
        "METS": 2,
        "activityGroupId": 5,
        "description": "breastfeeding, sitting or reclining"
    },
    {
        "id": 189,
        "code": "05190",
        "METS": 2.5,
        "activityGroupId": 5,
        "description": "sit, playing with animals, light effort, only active "
                       "periods"
    },
    {
        "id": 190,
        "code": "05191",
        "METS": 2.8,
        "activityGroupId": 5,
        "description": "stand, playing with animals, light effort, "
                       "only active periods"
    },
    {
        "id": 191,
        "code": "05192",
        "METS": 3,
        "activityGroupId": 5,
        "description": "walk/run, playing with animals, general, "
                       "light effort, only active periods"
    },
    {
        "id": 192,
        "code": "05193",
        "METS": 4,
        "activityGroupId": 5,
        "description": "walk/run, playing with animals, moderate effort, "
                       "only active periods"
    },
    {
        "id": 193,
        "code": "05194",
        "METS": 5,
        "activityGroupId": 5,
        "description": "walk/run, playing with animals, vigorous effort, "
                       "only active periods"
    },
    {
        "id": 194,
        "code": "05195",
        "METS": 3.5,
        "activityGroupId": 5,
        "description": "standing, bathing dog"
    },
    {
        "id": 195,
        "code": "05197",
        "METS": 2.3,
        "activityGroupId": 5,
        "description": "animal care, household animals, general"
    },
    {
        "id": 196,
        "code": "05200",
        "METS": 4,
        "activityGroupId": 5,
        "description": "elder care, disabled adult, bathing, dressing, "
                       "moving into and out of bed, only active periods"
    },
    {
        "id": 197,
        "code": "05205",
        "METS": 2.3,
        "activityGroupId": 5,
        "description": "elder care, disabled adult, feeding, combing hair, "
                       "light effort, only active periods"
    },
    {
        "id": 198,
        "code": "06010",
        "METS": 3,
        "activityGroupId": 6,
        "description": "airplane repair"
    },
    {
        "id": 199,
        "code": "06020",
        "METS": 4,
        "activityGroupId": 6,
        "description": "automobile body work"
    },
    {
        "id": 200,
        "code": "06030",
        "METS": 3.3,
        "activityGroupId": 6,
        "description": "automobile repair, light or moderate effort"
    },
    {
        "id": 201,
        "code": "06040",
        "METS": 3,
        "activityGroupId": 6,
        "description": "carpentry, general, workshop (Taylor code 620)"
    },
    {
        "id": 202,
        "code": "06050",
        "METS": 6,
        "activityGroupId": 6,
        "description": "carpentry, outside house, installing rain gutters ("
                       "Taylor code 640),carpentry, outside house, building a"
                       " fence"
    },
    {
        "id": 203,
        "code": "06052",
        "METS": 3.8,
        "activityGroupId": 6,
        "description": "carpentry, outside house, building a fence"
    },
    {
        "id": 204,
        "code": "06060",
        "METS": 3.3,
        "activityGroupId": 6,
        "description": "carpentry, finishing or refinishing cabinets or "
                       "furniture"
    },
    {
        "id": 205,
        "code": "06070",
        "METS": 6,
        "activityGroupId": 6,
        "description": "carpentry, sawing hardwood"
    },
    {
        "id": 206,
        "code": "06072",
        "METS": 4,
        "activityGroupId": 6,
        "description": "carpentry, home remodeling tasks, moderate effort"
    },
    {
        "id": 207,
        "code": "06074",
        "METS": 2.3,
        "activityGroupId": 6,
        "description": "carpentry, home remodeling tasks, light effort"
    },
    {
        "id": 208,
        "code": "06080",
        "METS": 5,
        "activityGroupId": 6,
        "description": "caulking, chinking log cabin"
    },
    {
        "id": 209,
        "code": "06090",
        "METS": 4.5,
        "activityGroupId": 6,
        "description": "caulking, except log cabin"
    },
    {
        "id": 210,
        "code": "06100",
        "METS": 5,
        "activityGroupId": 6,
        "description": "cleaning gutters"
    },
    {
        "id": 211,
        "code": "06110",
        "METS": 5,
        "activityGroupId": 6,
        "description": "excavating garage"
    },
    {
        "id": 212,
        "code": "06120",
        "METS": 5,
        "activityGroupId": 6,
        "description": "hanging storm windows"
    },
    {
        "id": 213,
        "code": "06122",
        "METS": 5,
        "activityGroupId": 6,
        "description": "hanging sheet rock inside house"
    },
    {
        "id": 214,
        "code": "06124",
        "METS": 3,
        "activityGroupId": 6,
        "description": "hammering nails"
    },
    {
        "id": 215,
        "code": "06126",
        "METS": 2.5,
        "activityGroupId": 6,
        "description": "home repair, general, light effort"
    },
    {
        "id": 216,
        "code": "06127",
        "METS": 4.5,
        "activityGroupId": 6,
        "description": "home repair, general, moderate effort"
    },
    {
        "id": 217,
        "code": "06128",
        "METS": 6,
        "activityGroupId": 6,
        "description": "home repair, general, vigorous effort"
    },
    {
        "id": 218,
        "code": "06130",
        "METS": 4.5,
        "activityGroupId": 6,
        "description": "laying or removing carpet"
    },
    {
        "id": 219,
        "code": "06140",
        "METS": 3.8,
        "activityGroupId": 6,
        "description": "laying tile or linoleum,repairing appliances"
    },
    {
        "id": 220,
        "code": "06144",
        "METS": 3,
        "activityGroupId": 6,
        "description": "repairing appliances"
    },
    {
        "id": 221,
        "code": "06150",
        "METS": 5,
        "activityGroupId": 6,
        "description": "painting, outside home (Taylor code 650)"
    },
    {
        "id": 222,
        "code": "06160",
        "METS": 3.3,
        "activityGroupId": 6,
        "description": "painting inside house,wallpapering, scraping paint"
    },
    {
        "id": 223,
        "code": "06165",
        "METS": 4.5,
        "activityGroupId": 6,
        "description": "painting, (Taylor code 630)"
    },
    {
        "id": 224,
        "code": "06167",
        "METS": 3,
        "activityGroupId": 6,
        "description": "plumbing, general"
    },
    {
        "id": 225,
        "code": "06170",
        "METS": 3,
        "activityGroupId": 6,
        "description": "put on and removal of tarp - sailboat"
    },
    {
        "id": 226,
        "code": "06180",
        "METS": 6,
        "activityGroupId": 6,
        "description": "roofing"
    },
    {
        "id": 227,
        "code": "06190",
        "METS": 4.5,
        "activityGroupId": 6,
        "description": "sanding floors with a power sander"
    },
    {
        "id": 228,
        "code": "06200",
        "METS": 4.5,
        "activityGroupId": 6,
        "description": "scraping and painting sailboat or powerboat"
    },
    {
        "id": 229,
        "code": "06205",
        "METS": 2,
        "activityGroupId": 6,
        "description": "sharpening tools"
    },
    {
        "id": 230,
        "code": "06210",
        "METS": 5,
        "activityGroupId": 6,
        "description": "spreading dirt with a shovel"
    },
    {
        "id": 231,
        "code": "06220",
        "METS": 4.5,
        "activityGroupId": 6,
        "description": "washing and waxing hull of sailboat or airplane"
    },
    {
        "id": 232,
        "code": "06225",
        "METS": 2,
        "activityGroupId": 6,
        "description": "washing and waxing car"
    },
    {
        "id": 233,
        "code": "06230",
        "METS": 4.5,
        "activityGroupId": 6,
        "description": "washing fence, painting fence, moderate effort"
    },
    {
        "id": 234,
        "code": "06240",
        "METS": 3.3,
        "activityGroupId": 6,
        "description": "wiring, tapping-splicing"
    },
    {
        "id": 235,
        "code": "07010",
        "METS": 1,
        "activityGroupId": 7,
        "description": "lying quietly and watching television"
    },
    {
        "id": 236,
        "code": "07011",
        "METS": 1.3,
        "activityGroupId": 7,
        "description": "lying quietly, doing nothing, lying in bed awake, "
                       "listening to music (not talking or reading)"
    },
    {
        "id": 237,
        "code": "07020",
        "METS": 1.3,
        "activityGroupId": 7,
        "description": "sitting quietly and watching television"
    },
    {
        "id": 238,
        "code": "07021",
        "METS": 1.3,
        "activityGroupId": 7,
        "description": "sitting quietly, general"
    },
    {
        "id": 239,
        "code": "07022",
        "METS": 1.5,
        "activityGroupId": 7,
        "description": "sitting quietly, fidgeting, general, fidgeting hands"
    },
    {
        "id": 240,
        "code": "07023",
        "METS": 1.8,
        "activityGroupId": 7,
        "description": "sitting, fidgeting feet"
    },
    {
        "id": 241,
        "code": "07024",
        "METS": 1.3,
        "activityGroupId": 7,
        "description": "sitting, smoking"
    },
    {
        "id": 242,
        "code": "07025",
        "METS": 1.5,
        "activityGroupId": 7,
        "description": "sitting, listening to music (not talking or reading) "
                       "or watching a movie in a theater"
    },
    {
        "id": 243,
        "code": "07026",
        "METS": 1.3,
        "activityGroupId": 7,
        "description": "sitting at a desk, resting head in hands"
    },
    {
        "id": 244,
        "code": "07030",
        "METS": 1,
        "activityGroupId": 7,
        "description": "sleeping"
    },
    {
        "id": 245,
        "code": "07040",
        "METS": 1.3,
        "activityGroupId": 7,
        "description": "standing quietly, standing in a line"
    },
    {
        "id": 246,
        "code": "07041",
        "METS": 1.8,
        "activityGroupId": 7,
        "description": "standing, fidgeting"
    },
    {
        "id": 247,
        "code": "07050",
        "METS": 1.3,
        "activityGroupId": 7,
        "description": "reclining, writing"
    },
    {
        "id": 248,
        "code": "07060",
        "METS": 1.3,
        "activityGroupId": 7,
        "description": "reclining, talking or talking on phone"
    },
    {
        "id": 249,
        "code": "07070",
        "METS": 1.3,
        "activityGroupId": 7,
        "description": "reclining, reading"
    },
    {
        "id": 250,
        "code": "07075",
        "METS": 1,
        "activityGroupId": 7,
        "description": "meditating"
    },
    {
        "id": 251,
        "code": "08009",
        "METS": 3.3,
        "activityGroupId": 8,
        "description": "carrying, loading or stacking wood, loading/unloading "
                       "or carrying lumber, light-to-moderate effort"
    },
    {
        "id": 252,
        "code": "08010",
        "METS": 5.5,
        "activityGroupId": 8,
        "description": "carrying, loading or stacking wood, loading/unloading "
                       "or carrying lumber"
    },
    {
        "id": 253,
        "code": "08019",
        "METS": 4.5,
        "activityGroupId": 8,
        "description": "chopping wood, splitting logs, moderate effort"
    },
    {
        "id": 254,
        "code": "08020",
        "METS": 6.3,
        "activityGroupId": 8,
        "description": "chopping wood, splitting logs, vigorous effort"
    },
    {
        "id": 255,
        "code": "08025",
        "METS": 3.5,
        "activityGroupId": 8,
        "description": "clearing light brush, thinning garden, moderate effort"
    },
    {
        "id": 256,
        "code": "08030",
        "METS": 6.3,
        "activityGroupId": 8,
        "description": "clearing brush/land, undergrowth, or ground, hauling "
                       "branches, wheelbarrow chores, vigorous effort"
    },
    {
        "id": 257,
        "code": "08040",
        "METS": 5,
        "activityGroupId": 8,
        "description": "digging sandbox, shoveling sand"
    },
    {
        "id": 258,
        "code": "08045",
        "METS": 3.5,
        "activityGroupId": 8,
        "description": "digging, spading, filling garden, composting, "
                       "light-to-moderate effort"
    },
    {
        "id": 259,
        "code": "08050",
        "METS": 5,
        "activityGroupId": 8,
        "description": "digging, spading, filling garden, compositing, "
                       "(Taylor code 590)"
    },
    {
        "id": 260,
        "code": "08052",
        "METS": 7.8,
        "activityGroupId": 8,
        "description": "digging, spading, filling garden, composting, "
                       "vigorous effort"
    },
    {
        "id": 261,
        "code": "08055",
        "METS": 2.8,
        "activityGroupId": 8,
        "description": "driving tractor"
    },
    {
        "id": 262,
        "code": "08057",
        "METS": 8.3,
        "activityGroupId": 8,
        "description": "felling trees, large size"
    },
    {
        "id": 263,
        "code": "08058",
        "METS": 5.3,
        "activityGroupId": 8,
        "description": "felling trees, small-medium size"
    },
    {
        "id": 264,
        "code": "08060",
        "METS": 5.8,
        "activityGroupId": 8,
        "description": "gardening with heavy power tools, tilling a garden, "
                       "chain saw"
    },
    {
        "id": 265,
        "code": "08065",
        "METS": 2.3,
        "activityGroupId": 8,
        "description": "gardening, using containers, older adults > 60 years"
    },
    {
        "id": 266,
        "code": "08070",
        "METS": 4,
        "activityGroupId": 8,
        "description": "irrigation channels, opening and closing ports"
    },
    {
        "id": 267,
        "code": "08080",
        "METS": 6.3,
        "activityGroupId": 8,
        "description": "laying crushed rock"
    },
    {
        "id": 268,
        "code": "08090",
        "METS": 5,
        "activityGroupId": 8,
        "description": "laying sod"
    },
    {
        "id": 269,
        "code": "08095",
        "METS": 5.5,
        "activityGroupId": 8,
        "description": "mowing lawn, general"
    },
    {
        "id": 270,
        "code": "08100",
        "METS": 2.5,
        "activityGroupId": 8,
        "description": "mowing lawn, riding mower (Taylor code 550)"
    },
    {
        "id": 271,
        "code": "08110",
        "METS": 6,
        "activityGroupId": 8,
        "description": "mowing lawn, walk, hand mower (Taylor code 570)"
    },
    {
        "id": 272,
        "code": "08120",
        "METS": 5,
        "activityGroupId": 8,
        "description": "mowing lawn, walk, power mower, moderate or vigorous "
                       "effort"
    },
    {
        "id": 273,
        "code": "08125",
        "METS": 4.5,
        "activityGroupId": 8,
        "description": "mowing lawn, power mower, light or moderate effort ("
                       "Taylor code 590)"
    },
    {
        "id": 274,
        "code": "08130",
        "METS": 2.5,
        "activityGroupId": 8,
        "description": "operating snow blower, walking"
    },
    {
        "id": 275,
        "code": "08135",
        "METS": 2,
        "activityGroupId": 8,
        "description": "planting, potting, transplanting seedlings or plants,"
                       " light effort"
    },
    {
        "id": 276,
        "code": "08140",
        "METS": 4.3,
        "activityGroupId": 8,
        "description": "planting seedlings, shrub, stooping, moderate effort"
    },
    {
        "id": 277,
        "code": "08145",
        "METS": 4.3,
        "activityGroupId": 8,
        "description": "planting crops or garden, stooping, moderate effort"
    },
    {
        "id": 278,
        "code": "08150",
        "METS": 4.5,
        "activityGroupId": 8,
        "description": "planting trees"
    },
    {
        "id": 279,
        "code": "08160",
        "METS": 3.8,
        "activityGroupId": 8,
        "description": "raking lawn or leaves, moderate effort"
    },
    {
        "id": 280,
        "code": "08165",
        "METS": 4,
        "activityGroupId": 8,
        "description": "raking lawn (Taylor code 600)"
    },
    {
        "id": 281,
        "code": "08170",
        "METS": 4,
        "activityGroupId": 8,
        "description": "raking roof with snow rake"
    },
    {
        "id": 282,
        "code": "08180",
        "METS": 3,
        "activityGroupId": 8,
        "description": "riding snow blower"
    },
    {
        "id": 283,
        "code": "08190",
        "METS": 4,
        "activityGroupId": 8,
        "description": "sacking grass, leaves"
    },
    {
        "id": 284,
        "code": "08192",
        "METS": 5.5,
        "activityGroupId": 8,
        "description": "shoveling dirt or mud"
    },
    {
        "id": 285,
        "code": "08195",
        "METS": 5.3,
        "activityGroupId": 8,
        "description": "shoveling snow, by hand, moderate effort"
    },
    {
        "id": 286,
        "code": "08200",
        "METS": 6,
        "activityGroupId": 8,
        "description": "shovelling snow, by hand (Taylor code 610)"
    },
    {
        "id": 287,
        "code": "08202",
        "METS": 7.5,
        "activityGroupId": 8,
        "description": "shoveling snow, by hand, vigorous effort"
    },
    {
        "id": 288,
        "code": "08210",
        "METS": 4,
        "activityGroupId": 8,
        "description": "trimming shrubs or trees, manual cutter"
    },
    {
        "id": 289,
        "code": "08215",
        "METS": 3.5,
        "activityGroupId": 8,
        "description": "trimming shrubs or trees, power cutter, using leaf "
                       "blower, edge, moderate effort"
    },
    {
        "id": 290,
        "code": "08220",
        "METS": 3,
        "activityGroupId": 8,
        "description": "walking, applying fertilizer or seeding a lawn, "
                       "push applicator"
    },
    {
        "id": 291,
        "code": "08230",
        "METS": 1.5,
        "activityGroupId": 8,
        "description": "watering lawn or garden, standing or walking"
    },
    {
        "id": 292,
        "code": "08239",
        "METS": 3.5,
        "activityGroupId": 8,
        "description": "weeding, cultivating garden, light-to-moderate effort"
    },
    {
        "id": 293,
        "code": "08240",
        "METS": 4.5,
        "activityGroupId": 8,
        "description": "weeding, cultivating garden (Taylor code 580)"
    },
    {
        "id": 294,
        "code": "08241",
        "METS": 5,
        "activityGroupId": 8,
        "description": "weeding, cultivating garden, using a hoe, "
                       "moderate-to-vigorous effort"
    },
    {
        "id": 295,
        "code": "08245",
        "METS": 3.8,
        "activityGroupId": 8,
        "description": "gardening, general, moderate effort"
    },
    {
        "id": 296,
        "code": "08246",
        "METS": 3.5,
        "activityGroupId": 8,
        "description": "picking fruit off trees, picking fruits/vegetables, "
                       "moderate effort"
    },
    {
        "id": 297,
        "code": "08248",
        "METS": 4.5,
        "activityGroupId": 8,
        "description": "picking fruit off trees, gleaning fruits, picking "
                       "fruits/vegetables, climbing ladder to pick fruit, "
                       "vigorous effort"
    },
    {
        "id": 298,
        "code": "08250",
        "METS": 3.3,
        "activityGroupId": 8,
        "description": "implied walking/standing - picking up yard, light, "
                       "picking flowers or vegetables"
    },
    {
        "id": 299,
        "code": "08251",
        "METS": 3,
        "activityGroupId": 8,
        "description": "walking, gathering gardening tools"
    },
    {
        "id": 300,
        "code": "08255",
        "METS": 5.5,
        "activityGroupId": 8,
        "description": "wheelbarrow, pushing garden cart or wheelbarrow"
    },
    {
        "id": 301,
        "code": "08260",
        "METS": 3,
        "activityGroupId": 8,
        "description": "yard work, general, light effort"
    },
    {
        "id": 302,
        "code": "08261",
        "METS": 4,
        "activityGroupId": 8,
        "description": "yard work, general, moderate effort"
    },
    {
        "id": 303,
        "code": "08262",
        "METS": 6,
        "activityGroupId": 8,
        "description": "yard work, general, vigorous effort"
    },
    {
        "id": 304,
        "code": "09000",
        "METS": 1.5,
        "activityGroupId": 9,
        "description": "board game playing, sitting"
    },
    {
        "id": 305,
        "code": "09005",
        "METS": 2.5,
        "activityGroupId": 9,
        "description": "casino gambling, standing"
    },
    {
        "id": 306,
        "code": "09010",
        "METS": 1.5,
        "activityGroupId": 9,
        "description": "card playing, sitting"
    },
    {
        "id": 307,
        "code": "09013",
        "METS": 1.5,
        "activityGroupId": 9,
        "description": "chess game, sitting"
    },
    {
        "id": 308,
        "code": "09015",
        "METS": 1.5,
        "activityGroupId": 9,
        "description": "copying documents, standing"
    },
    {
        "id": 309,
        "code": "09020",
        "METS": 1.8,
        "activityGroupId": 9,
        "description": "drawing, writing, painting, standing"
    },
    {
        "id": 310,
        "code": "09025",
        "METS": 1,
        "activityGroupId": 9,
        "description": "laughing, sitting"
    },
    {
        "id": 311,
        "code": "09030",
        "METS": 1.3,
        "activityGroupId": 9,
        "description": "sitting, reading, book, newspaper, etc."
    },
    {
        "id": 312,
        "code": "09040",
        "METS": 1.3,
        "activityGroupId": 9,
        "description": "sitting, writing, desk work, typing"
    },
    {
        "id": 313,
        "code": "09045",
        "METS": 1,
        "activityGroupId": 9,
        "description": "sitting, playing traditional video game, computer game"
    },
    {
        "id": 314,
        "code": "09050",
        "METS": 1.8,
        "activityGroupId": 9,
        "description": "standing, talking in person, on the phone, computer, "
                       "or text messaging, light effort"
    },
    {
        "id": 315,
        "code": "09055",
        "METS": 1.5,
        "activityGroupId": 9,
        "description": "sitting, talking in person, on the phone, computer, "
                       "or text messaging, light effort"
    },
    {
        "id": 316,
        "code": "09060",
        "METS": 1.3,
        "activityGroupId": 9,
        "description": "sitting, studying, general, including reading and/or "
                       "writing, light effort"
    },
    {
        "id": 317,
        "code": "09065",
        "METS": 1.8,
        "activityGroupId": 9,
        "description": "sitting, in class, general, including note-taking or "
                       "class discussion"
    },
    {
        "id": 318,
        "code": "09070",
        "METS": 1.8,
        "activityGroupId": 9,
        "description": "standing, reading"
    },
    {
        "id": 319,
        "code": "09071",
        "METS": 2.5,
        "activityGroupId": 9,
        "description": "standing, miscellaneous"
    },
    {
        "id": 320,
        "code": "09075",
        "METS": 1.8,
        "activityGroupId": 9,
        "description": "sitting, arts and crafts, carving wood, weaving, "
                       "spinning wool, light effort"
    },
    {
        "id": 321,
        "code": "09080",
        "METS": 3,
        "activityGroupId": 9,
        "description": "sitting, arts and crafts, carving wood, weaving, "
                       "spinning wool, moderate effort"
    },
    {
        "id": 322,
        "code": "09085",
        "METS": 2.5,
        "activityGroupId": 9,
        "description": "standing, arts and crafts, sand painting, carving, "
                       "weaving, light effort"
    },
    {
        "id": 323,
        "code": "09090",
        "METS": 3.3,
        "activityGroupId": 9,
        "description": "standing, arts and crafts, sand painting, carving, "
                       "weaving, moderate effort"
    },
    {
        "id": 324,
        "code": "09095",
        "METS": 3.5,
        "activityGroupId": 9,
        "description": "standing, arts and crafts, sand painting, carving, "
                       "weaving, vigorous effort"
    },
    {
        "id": 325,
        "code": "09100",
        "METS": 1.8,
        "activityGroupId": 9,
        "description": "retreat/family reunion activities involving sitting, "
                       "relaxing, talking, eating"
    },
    {
        "id": 326,
        "code": "09101",
        "METS": 3,
        "activityGroupId": 9,
        "description": "retreat/family reunion activities involving playing "
                       "games with children"
    },
    {
        "id": 327,
        "code": "09105",
        "METS": 2,
        "activityGroupId": 9,
        "description": "touring/traveling/vacation involving riding in a "
                       "vehicle"
    },
    {
        "id": 328,
        "code": "09106",
        "METS": 3.5,
        "activityGroupId": 9,
        "description": "touring/traveling/vacation involving walking"
    },
    {
        "id": 329,
        "code": "09110",
        "METS": 2.5,
        "activityGroupId": 9,
        "description": "camping involving standing, walking, sitting, "
                       "light-to-moderate effort"
    },
    {
        "id": 330,
        "code": "09115",
        "METS": 1.5,
        "activityGroupId": 9,
        "description": "sitting at a sporting event, spectator"
    },
    {
        "id": 331,
        "code": "10010",
        "METS": 1.8,
        "activityGroupId": 10,
        "description": "accordion, sitting"
    },
    {
        "id": 332,
        "code": "10020",
        "METS": 2.3,
        "activityGroupId": 10,
        "description": "cello, sitting"
    },
    {
        "id": 333,
        "code": "10030",
        "METS": 2.3,
        "activityGroupId": 10,
        "description": "conducting orchestra, standing"
    },
    {
        "id": 334,
        "code": "10035",
        "METS": 2.5,
        "activityGroupId": 10,
        "description": "double bass, standing"
    },
    {
        "id": 335,
        "code": "10040",
        "METS": 3.8,
        "activityGroupId": 10,
        "description": "drums, sitting"
    },
    {
        "id": 336,
        "code": "10045",
        "METS": 3,
        "activityGroupId": 10,
        "description": "drumming (e.g., bongo, conga, benbe), moderate, sitting"
    },
    {
        "id": 337,
        "code": "10050",
        "METS": 2,
        "activityGroupId": 10,
        "description": "flute, sitting"
    },
    {
        "id": 338,
        "code": "10060",
        "METS": 1.8,
        "activityGroupId": 10,
        "description": "horn, standing"
    },
    {
        "id": 339,
        "code": "10070",
        "METS": 2.3,
        "activityGroupId": 10,
        "description": "piano, sitting"
    },
    {
        "id": 340,
        "code": "10074",
        "METS": 2,
        "activityGroupId": 10,
        "description": "playing musical instruments, general"
    },
    {
        "id": 341,
        "code": "10077",
        "METS": 2,
        "activityGroupId": 10,
        "description": "organ, sitting"
    },
    {
        "id": 342,
        "code": "10080",
        "METS": 3.5,
        "activityGroupId": 10,
        "description": "trombone, standing"
    },
    {
        "id": 343,
        "code": "10090",
        "METS": 1.8,
        "activityGroupId": 10,
        "description": "trumpet, standing"
    },
    {
        "id": 344,
        "code": "10100",
        "METS": 2.5,
        "activityGroupId": 10,
        "description": "violin, sitting"
    },
    {
        "id": 345,
        "code": "10110",
        "METS": 1.8,
        "activityGroupId": 10,
        "description": "woodwind, sitting"
    },
    {
        "id": 346,
        "code": "10120",
        "METS": 2,
        "activityGroupId": 10,
        "description": "guitar, classical, folk, sitting"
    },
    {
        "id": 347,
        "code": "10125",
        "METS": 3,
        "activityGroupId": 10,
        "description": "guitar, rock and roll band, standing"
    },
    {
        "id": 348,
        "code": "10130",
        "METS": 4,
        "activityGroupId": 10,
        "description": "marching band, baton twirling, walking, moderate "
                       "pace, general"
    },
    {
        "id": 349,
        "code": "10131",
        "METS": 5.5,
        "activityGroupId": 10,
        "description": "marching band, playing an instrument, walking, "
                       "brisk pace, general"
    },
    {
        "id": 350,
        "code": "10135",
        "METS": 3.5,
        "activityGroupId": 10,
        "description": "marching band, drum major, walking"
    },
    {
        "id": 351,
        "code": "11003",
        "METS": 2.3,
        "activityGroupId": 11,
        "description": "active workstation, treadmill desk, walking"
    },
    {
        "id": 352,
        "code": "11006",
        "METS": 3,
        "activityGroupId": 11,
        "description": "airline flight attendant"
    },
    {
        "id": 353,
        "code": "11010",
        "METS": 4,
        "activityGroupId": 11,
        "description": "bakery, general, moderate effort"
    },
    {
        "id": 354,
        "code": "11015",
        "METS": 2,
        "activityGroupId": 11,
        "description": "bakery, light effort"
    },
    {
        "id": 355,
        "code": "11020",
        "METS": 2.3,
        "activityGroupId": 11,
        "description": "bookbinding"
    },
    {
        "id": 356,
        "code": "11030",
        "METS": 6,
        "activityGroupId": 11,
        "description": "building road, driving heavy machinery"
    },
    {
        "id": 357,
        "code": "11035",
        "METS": 2,
        "activityGroupId": 11,
        "description": "building road, directing traffic, standing"
    },
    {
        "id": 358,
        "code": "11038",
        "METS": 2.5,
        "activityGroupId": 11,
        "description": "carpentry, general, light effort"
    },
    {
        "id": 359,
        "code": "11040",
        "METS": 4.3,
        "activityGroupId": 11,
        "description": "carpentry, general, moderate effort"
    },
    {
        "id": 360,
        "code": "11042",
        "METS": 7,
        "activityGroupId": 11,
        "description": "carpentry, general, heavy or vigorous effort"
    },
    {
        "id": 361,
        "code": "11050",
        "METS": 8,
        "activityGroupId": 11,
        "description": "carrying heavy loads (e.g., bricks, tools)"
    },
    {
        "id": 362,
        "code": "11060",
        "METS": 8,
        "activityGroupId": 11,
        "description": "carrying moderate loads up stairs, moving boxes 25-49"
                       " lbs"
    },
    {
        "id": 363,
        "code": "11070",
        "METS": 4,
        "activityGroupId": 11,
        "description": "chambermaid, hotel housekeeper, making bed, cleaning "
                       "bathroom, pushing cart"
    },
    {
        "id": 364,
        "code": "11080",
        "METS": 5.3,
        "activityGroupId": 11,
        "description": "coal mining, drilling coal, rock"
    },
    {
        "id": 365,
        "code": "11090",
        "METS": 5,
        "activityGroupId": 11,
        "description": "coal mining, erecting supports"
    },
    {
        "id": 366,
        "code": "11100",
        "METS": 5.5,
        "activityGroupId": 11,
        "description": "coal mining, general"
    },
    {
        "id": 367,
        "code": "11110",
        "METS": 6.3,
        "activityGroupId": 11,
        "description": "coal mining, shoveling coal"
    },
    {
        "id": 368,
        "code": "11115",
        "METS": 2.5,
        "activityGroupId": 11,
        "description": "cook, chef"
    },
    {
        "id": 369,
        "code": "11120",
        "METS": 4,
        "activityGroupId": 11,
        "description": "construction, outside, remodeling, new structures ("
                       "e.g., roof repair, miscellaneous)"
    },
    {
        "id": 370,
        "code": "11125",
        "METS": 2.3,
        "activityGroupId": 11,
        "description": "custodial work, light effort (e.g., cleaning sink and "
                       "toilet, dusting, vacuuming, light cleaning)"
    },
    {
        "id": 371,
        "code": "11126",
        "METS": 3.8,
        "activityGroupId": 11,
        "description": "custodial work, moderate effort (e.g., electric "
                       "buffer, feathering arena floors, mopping, taking out "
                       "trash, vacuuming)"
    },
    {
        "id": 372,
        "code": "11130",
        "METS": 3.3,
        "activityGroupId": 11,
        "description": "electrical work (e.g., hook up wire, tapping-splicing)"
    },
    {
        "id": 373,
        "code": "11135",
        "METS": 1.8,
        "activityGroupId": 11,
        "description": "engineer (e.g., mechanical or electrical)"
    },
    {
        "id": 374,
        "code": "11145",
        "METS": 7.8,
        "activityGroupId": 11,
        "description": "farming, vigorous effort (e.g., baling hay, cleaning "
                       "barn)"
    },
    {
        "id": 375,
        "code": "11146",
        "METS": 4.8,
        "activityGroupId": 11,
        "description": "farming, moderate effort (e.g., feeding animals, "
                       "chasing cattle by walking and/or horseback, spreading "
                       "manure, harvesting crops)"
    },
    {
        "id": 376,
        "code": "11147",
        "METS": 2,
        "activityGroupId": 11,
        "description": "farming, light effort (e.g., cleaning animal sheds, "
                       "preparing animal feed)"
    },
    {
        "id": 377,
        "code": "11170",
        "METS": 2.8,
        "activityGroupId": 11,
        "description": "farming, driving tasks (e.g., driving tractor or "
                       "harvester)"
    },
    {
        "id": 378,
        "code": "11180",
        "METS": 3.5,
        "activityGroupId": 11,
        "description": "farming, feeding small animals"
    },
    {
        "id": 379,
        "code": "11190",
        "METS": 4.3,
        "activityGroupId": 11,
        "description": "farming, feeding cattle, horses"
    },
    {
        "id": 380,
        "code": "11191",
        "METS": 4.3,
        "activityGroupId": 11,
        "description": "farming, hauling water for animals, general hauling "
                       "water,farming, general hauling water"
    },
    {
        "id": 381,
        "code": "11192",
        "METS": 4.5,
        "activityGroupId": 11,
        "description": "farming, taking care of animals (e.g., grooming, "
                       "brushing, shearing sheep, assisting with birthing, "
                       "medical care, branding), general"
    },
    {
        "id": 382,
        "code": "11195",
        "METS": 3.8,
        "activityGroupId": 11,
        "description": "farming, rice, planting, grain milling activities"
    },
    {
        "id": 383,
        "code": "11210",
        "METS": 3.5,
        "activityGroupId": 11,
        "description": "farming, milking by hand, cleaning pails, moderate "
                       "effort"
    },
    {
        "id": 384,
        "code": "11220",
        "METS": 1.3,
        "activityGroupId": 11,
        "description": "farming, milking by machine, light effort"
    },
    {
        "id": 385,
        "code": "11240",
        "METS": 8,
        "activityGroupId": 11,
        "description": "fire fighter, general"
    },
    {
        "id": 386,
        "code": "11244",
        "METS": 6.8,
        "activityGroupId": 11,
        "description": "fire fighter, rescue victim, automobile accident, "
                       "using pike pole"
    },
    {
        "id": 387,
        "code": "11245",
        "METS": 8,
        "activityGroupId": 11,
        "description": "fire fighter, raising and climbing ladder with full "
                       "gear, simulated fire supression"
    },
    {
        "id": 388,
        "code": "11246",
        "METS": 9,
        "activityGroupId": 11,
        "description": "fire fighter, hauling hoses on ground, "
                       "carrying/hoisting equipment, breaking down walls "
                       "etc., wearing full gear"
    },
    {
        "id": 389,
        "code": "11247",
        "METS": 3.5,
        "activityGroupId": 11,
        "description": "fishing, commercial, light effort"
    },
    {
        "id": 390,
        "code": "11248",
        "METS": 5,
        "activityGroupId": 11,
        "description": "fishing, commercial, moderate effort"
    },
    {
        "id": 391,
        "code": "11249",
        "METS": 7,
        "activityGroupId": 11,
        "description": "fishing, commercial, vigorous effort"
    },
    {
        "id": 392,
        "code": "11250",
        "METS": 17.5,
        "activityGroupId": 11,
        "description": "forestry, ax chopping, very fast, 1.25 kg axe, "
                       "51 blows/min, extremely vigorous effort"
    },
    {
        "id": 393,
        "code": "11260",
        "METS": 5,
        "activityGroupId": 11,
        "description": "forestry, ax chopping, slow, 1.25 kg axe, "
                       "19 blows/min, moderate effort"
    },
    {
        "id": 394,
        "code": "11262",
        "METS": 8,
        "activityGroupId": 11,
        "description": "forestry, ax chopping, fast, 1.25 kg axe, "
                       "35 blows/min, vigorous effort"
    },
    {
        "id": 395,
        "code": "11264",
        "METS": 4.5,
        "activityGroupId": 11,
        "description": "forestry, moderate effort (e.g., sawing wood with "
                       "power saw, weeding, hoeing)"
    },
    {
        "id": 396,
        "code": "11266",
        "METS": 8,
        "activityGroupId": 11,
        "description": "forestry, vigorous effort (e.g., barking, felling, "
                       "or trimming trees, carrying or stacking logs, "
                       "planting seeds, sawing lumber by hand)"
    },
    {
        "id": 397,
        "code": "11370",
        "METS": 4.5,
        "activityGroupId": 11,
        "description": "furriery"
    },
    {
        "id": 398,
        "code": "11375",
        "METS": 4,
        "activityGroupId": 11,
        "description": "garbage collector, walking, dumping bins into truck"
    },
    {
        "id": 399,
        "code": "11378",
        "METS": 1.8,
        "activityGroupId": 11,
        "description": "hairstylist (e.g., plaiting hair, manicure, make-up "
                       "artist)"
    },
    {
        "id": 400,
        "code": "11380",
        "METS": 7.3,
        "activityGroupId": 11,
        "description": "horse grooming, including feeding, cleaning stalls, "
                       "bathing, brushing, clipping, longeing and exercising "
                       "horses"
    },
    {
        "id": 401,
        "code": "11381",
        "METS": 4.3,
        "activityGroupId": 11,
        "description": "horse, feeding, watering, cleaning stalls, implied "
                       "walking and lifting loads"
    },
    {
        "id": 402,
        "code": "11390",
        "METS": 7.3,
        "activityGroupId": 11,
        "description": "horse racing, galloping"
    },
    {
        "id": 403,
        "code": "11400",
        "METS": 5.8,
        "activityGroupId": 11,
        "description": "horse racing, trotting"
    },
    {
        "id": 404,
        "code": "11410",
        "METS": 3.8,
        "activityGroupId": 11,
        "description": "horse racing, walking"
    },
    {
        "id": 405,
        "code": "11413",
        "METS": 3,
        "activityGroupId": 11,
        "description": "kitchen maid"
    },
    {
        "id": 406,
        "code": "11415",
        "METS": 4,
        "activityGroupId": 11,
        "description": "lawn keeper, yard work, general"
    },
    {
        "id": 407,
        "code": "11418",
        "METS": 3.3,
        "activityGroupId": 11,
        "description": "laundry worker"
    },
    {
        "id": 408,
        "code": "11420",
        "METS": 3,
        "activityGroupId": 11,
        "description": "locksmith"
    },
    {
        "id": 409,
        "code": "11430",
        "METS": 3,
        "activityGroupId": 11,
        "description": "machine tooling (e.g., machining, working sheet "
                       "metal, machine fitter, operating lathe, welding) "
                       "light-to-moderate effort"
    },
    {
        "id": 410,
        "code": "11450",
        "METS": 5,
        "activityGroupId": 11,
        "description": "Machine tooling, operating punch press, moderate effort"
    },
    {
        "id": 411,
        "code": "11472",
        "METS": 1.8,
        "activityGroupId": 11,
        "description": "manager, property"
    },
    {
        "id": 412,
        "code": "11475",
        "METS": 2.8,
        "activityGroupId": 11,
        "description": "manual or unskilled labor, general, light effort"
    },
    {
        "id": 413,
        "code": "11476",
        "METS": 4.5,
        "activityGroupId": 11,
        "description": "manual or unskilled labor, general, moderate effort"
    },
    {
        "id": 414,
        "code": "11477",
        "METS": 6.5,
        "activityGroupId": 11,
        "description": "manual or unskilled labor, general, vigorous effort"
    },
    {
        "id": 415,
        "code": "11480",
        "METS": 4.3,
        "activityGroupId": 11,
        "description": "masonry, concrete, moderate effort"
    },
    {
        "id": 416,
        "code": "11482",
        "METS": 2.5,
        "activityGroupId": 11,
        "description": "masonry, concrete, light effort"
    },
    {
        "id": 417,
        "code": "11485",
        "METS": 4,
        "activityGroupId": 11,
        "description": "massage therapist, standing"
    },
    {
        "id": 418,
        "code": "11490",
        "METS": 7.5,
        "activityGroupId": 11,
        "description": "moving, carrying or pushing heavy objects, 75 lbs or "
                       "more, only active time (e.g., desks, moving van work)"
    },
    {
        "id": 419,
        "code": "11495",
        "METS": 12,
        "activityGroupId": 11,
        "description": "skindiving or SCUBA diving as a frogman, Navy Seal"
    },
    {
        "id": 420,
        "code": "11500",
        "METS": 2.5,
        "activityGroupId": 11,
        "description": "operating heavy duty equipment, automated, not driving"
    },
    {
        "id": 421,
        "code": "11510",
        "METS": 4.5,
        "activityGroupId": 11,
        "description": "orange grove work, picking fruit"
    },
    {
        "id": 422,
        "code": "11514",
        "METS": 3.3,
        "activityGroupId": 11,
        "description": "painting,house, furniture, moderate effort"
    },
    {
        "id": 423,
        "code": "11516",
        "METS": 3,
        "activityGroupId": 11,
        "description": "plumbing activities"
    },
    {
        "id": 424,
        "code": "11520",
        "METS": 2,
        "activityGroupId": 11,
        "description": "printing, paper industry worker, standing"
    },
    {
        "id": 425,
        "code": "11525",
        "METS": 2.5,
        "activityGroupId": 11,
        "description": "police, directing traffic, standing"
    },
    {
        "id": 426,
        "code": "11526",
        "METS": 2.5,
        "activityGroupId": 11,
        "description": "police, driving a squad car, sitting"
    },
    {
        "id": 427,
        "code": "11527",
        "METS": 1.3,
        "activityGroupId": 11,
        "description": "police, riding in a squad car, sitting"
    },
    {
        "id": 428,
        "code": "11528",
        "METS": 4,
        "activityGroupId": 11,
        "description": "police, making an arrest, standing"
    },
    {
        "id": 429,
        "code": "11529",
        "METS": 2.3,
        "activityGroupId": 11,
        "description": "postal carrier, walking to deliver mail"
    },
    {
        "id": 430,
        "code": "11530",
        "METS": 2,
        "activityGroupId": 11,
        "description": "shoe repair, general"
    },
    {
        "id": 431,
        "code": "11540",
        "METS": 7.8,
        "activityGroupId": 11,
        "description": "shoveling, digging ditches"
    },
    {
        "id": 432,
        "code": "11550",
        "METS": 8.8,
        "activityGroupId": 11,
        "description": "shoveling, more than 16 lbs/minute, deep digging, "
                       "vigorous effort"
    },
    {
        "id": 433,
        "code": "11560",
        "METS": 5,
        "activityGroupId": 11,
        "description": "shoveling, less than 10 lbs/minute, moderate effort"
    },
    {
        "id": 434,
        "code": "11570",
        "METS": 6.5,
        "activityGroupId": 11,
        "description": "shoveling, 10 to 15 lbs/minute, vigorous effort"
    },
    {
        "id": 435,
        "code": "11580",
        "METS": 1.5,
        "activityGroupId": 11,
        "description": "sitting tasks, light effort (e.g., office work, "
                       "chemistry lab work, computer work, light assembly "
                       "repair, watch repair, reading, desk work)"
    },
    {
        "id": 436,
        "code": "11585",
        "METS": 1.5,
        "activityGroupId": 11,
        "description": "sitting meetings, light effort, general, and/or with "
                       "talking involved (e.g., eating at a business meeting)"
    },
    {
        "id": 437,
        "code": "11590",
        "METS": 2.5,
        "activityGroupId": 11,
        "description": "sitting tasks, moderate effort (e.g., pushing heavy "
                       "levers, riding mower/forklift, crane operation)"
    },
    {
        "id": 438,
        "code": "11593",
        "METS": 2.8,
        "activityGroupId": 11,
        "description": "sitting, teaching stretching or yoga, or light effort "
                       "exercise class"
    },
    {
        "id": 439,
        "code": "11600",
        "METS": 3,
        "activityGroupId": 11,
        "description": "standing tasks, light effort (e.g., bartending, "
                       "store clerk, assembling, filing, duplicating, "
                       "librarian, putting up a Christmas tree, standing and "
                       "talking at work, changing clothes when teaching "
                       "physical education, standing)"
    },
    {
        "id": 440,
        "code": "11610",
        "METS": 3,
        "activityGroupId": 11,
        "description": "standing, light/moderate effort (e.g., "
                       "assemble/repair heavy parts, welding,stocking parts,"
                       "auto repair,standing, packing boxes, nursing patient "
                       "care)"
    },
    {
        "id": 441,
        "code": "11615",
        "METS": 4.5,
        "activityGroupId": 11,
        "description": "standing, moderate effort, lifting items "
                       "continuously, 10 – 20 lbs, with limited walking or "
                       "resting"
    },
    {
        "id": 442,
        "code": "11620",
        "METS": 3.5,
        "activityGroupId": 11,
        "description": "standing, moderate effort, intermittent lifting 50 "
                       "lbs, hitch/twisting ropes"
    },
    {
        "id": 443,
        "code": "11630",
        "METS": 4.5,
        "activityGroupId": 11,
        "description": "standing, moderate/heavy tasks (e.g., lifting more "
                       "than 50 lbs, masonry, painting, paper hanging)"
    },
    {
        "id": 444,
        "code": "11708",
        "METS": 5.3,
        "activityGroupId": 11,
        "description": "steel mill, moderate effort (e.g., fettling, forging, "
                       "tipping molds)"
    },
    {
        "id": 445,
        "code": "11710",
        "METS": 8.3,
        "activityGroupId": 11,
        "description": "steel mill, vigorous effort (e.g., hand rolling, "
                       "merchant mill rolling, removing slag, tending furnace)"
    },
    {
        "id": 446,
        "code": "11720",
        "METS": 2.3,
        "activityGroupId": 11,
        "description": "tailoring, cutting fabric"
    },
    {
        "id": 447,
        "code": "11730",
        "METS": 2.5,
        "activityGroupId": 11,
        "description": "tailoring, general"
    },
    {
        "id": 448,
        "code": "11740",
        "METS": 1.8,
        "activityGroupId": 11,
        "description": "tailoring, hand sewing"
    },
    {
        "id": 449,
        "code": "11750",
        "METS": 2.5,
        "activityGroupId": 11,
        "description": "tailoring, machine sewing"
    },
    {
        "id": 450,
        "code": "11760",
        "METS": 3.5,
        "activityGroupId": 11,
        "description": "tailoring, pressing"
    },
    {
        "id": 451,
        "code": "11763",
        "METS": 2,
        "activityGroupId": 11,
        "description": "tailoring, weaving, light effort (e.g., finishing "
                       "operations, washing, dyeing, inspecting cloth, "
                       "counting yards, paperwork)"
    },
    {
        "id": 452,
        "code": "11765",
        "METS": 4,
        "activityGroupId": 11,
        "description": "tailoring, weaving, moderate effort (e.g., spinning "
                       "and weaving operations, delivering boxes of yam to "
                       "spinners, loading of warp bean, pinwinding, "
                       "conewinding, warping, cloth cutting)"
    },
    {
        "id": 453,
        "code": "11766",
        "METS": 6.5,
        "activityGroupId": 11,
        "description": "truck driving, loading and unloading truck, "
                       "tying down load, standing, walking and carrying heavy"
                       " loads"
    },
    {
        "id": 454,
        "code": "11767",
        "METS": 2,
        "activityGroupId": 11,
        "description": "Truch, driving delivery truck, taxi, shuttlebus, "
                       "school bus"
    },
    {
        "id": 455,
        "code": "11770",
        "METS": 1.3,
        "activityGroupId": 11,
        "description": "typing, electric, manual or computer"
    },
    {
        "id": 456,
        "code": "11780",
        "METS": 6.3,
        "activityGroupId": 11,
        "description": "using heavy power tools such as pneumatic tools ("
                       "e.g., jackhammers, drills)"
    },
    {
        "id": 457,
        "code": "11790",
        "METS": 8,
        "activityGroupId": 11,
        "description": "using heavy tools (not power) such as shovel, pick, "
                       "tunnel bar, spade"
    },
    {
        "id": 458,
        "code": "11791",
        "METS": 2,
        "activityGroupId": 11,
        "description": "walking on job, less than 2.0 mph, very slow speed, "
                       "in office or lab area"
    },
    {
        "id": 459,
        "code": "11792",
        "METS": 3.5,
        "activityGroupId": 11,
        "description": "walking on job, 3.0 mph, in office, moderate speed, "
                       "not carrying anything"
    },
    {
        "id": 460,
        "code": "11793",
        "METS": 4.3,
        "activityGroupId": 11,
        "description": "walking on job, 3.5 mph, in office, brisk speed, "
                       "not carrying anything"
    },
    {
        "id": 461,
        "code": "11795",
        "METS": 3.5,
        "activityGroupId": 11,
        "description": "walking on job, 2.5 mph, slow speed and carrying "
                       "light objects less than 25 lbs"
    },
    {
        "id": 462,
        "code": "11796",
        "METS": 3,
        "activityGroupId": 11,
        "description": "walking, gathering things at work, ready to leave"
    },
    {
        "id": 463,
        "code": "11797",
        "METS": 3.8,
        "activityGroupId": 11,
        "description": "walking, 2.5 mph, slow speed, carrying heavy objects "
                       "more than 25 lbs"
    },
    {
        "id": 464,
        "code": "11800",
        "METS": 4.5,
        "activityGroupId": 11,
        "description": "walking, 3.0 mph, moderately and carrying light "
                       "objects less than 25 lbs"
    },
    {
        "id": 465,
        "code": "11805",
        "METS": 3.5,
        "activityGroupId": 11,
        "description": "walking, pushing a wheelchair"
    },
    {
        "id": 466,
        "code": "11810",
        "METS": 4.8,
        "activityGroupId": 11,
        "description": "walking, 3.5 mph, briskly and carrying objects less "
                       "than 25 lbs"
    },
    {
        "id": 467,
        "code": "11820",
        "METS": 5,
        "activityGroupId": 11,
        "description": "walking or walk downstairs or standing, carrying "
                       "objects about 25 to 49 lbs"
    },
    {
        "id": 468,
        "code": "11830",
        "METS": 6.5,
        "activityGroupId": 11,
        "description": "walking or walk downstairs or standing, carrying "
                       "objects about 50 to 74 lbs"
    },
    {
        "id": 469,
        "code": "11840",
        "METS": 7.5,
        "activityGroupId": 11,
        "description": "walking or walk downstairs or standing, carrying "
                       "objects about 75 to 99 lbs"
    },
    {
        "id": 470,
        "code": "11850",
        "METS": 8.5,
        "activityGroupId": 11,
        "description": "walking or walk downstairs or standing, carrying "
                       "objects about 100 lbs or more"
    },
    {
        "id": 471,
        "code": "11870",
        "METS": 3,
        "activityGroupId": 11,
        "description": "working in scene shop, theater actor, backstage "
                       "employee"
    },
    {
        "id": 472,
        "code": "12010",
        "METS": 6,
        "activityGroupId": 12,
        "description": "jog/walk combination (jogging component of less than "
                       "10 minutes) (Taylor code 180)"
    },
    {
        "id": 473,
        "code": "12020",
        "METS": 7,
        "activityGroupId": 12,
        "description": "jogging, general"
    },
    {
        "id": 474,
        "code": "12025",
        "METS": 8,
        "activityGroupId": 12,
        "description": "jogging, in place"
    },
    {
        "id": 475,
        "code": "12027",
        "METS": 4.5,
        "activityGroupId": 12,
        "description": "jogging, on a mini-tramp"
    },
    {
        "id": 476,
        "code": "12029",
        "METS": 6,
        "activityGroupId": 12,
        "description": "Running, 4 mph (13 min/mile)"
    },
    {
        "id": 477,
        "code": "12030",
        "METS": 8.3,
        "activityGroupId": 12,
        "description": "running, 5 mph (12 min/mile)"
    },
    {
        "id": 478,
        "code": "12040",
        "METS": 9,
        "activityGroupId": 12,
        "description": "running, 5.2 mph (11.5 min/mile)"
    },
    {
        "id": 479,
        "code": "12050",
        "METS": 9.8,
        "activityGroupId": 12,
        "description": "running, 6 mph (10 min/mile)"
    },
    {
        "id": 480,
        "code": "12060",
        "METS": 10.5,
        "activityGroupId": 12,
        "description": "running, 6.7 mph (9 min/mile)"
    },
    {
        "id": 481,
        "code": "12070",
        "METS": 11,
        "activityGroupId": 12,
        "description": "running, 7 mph (8.5 min/mile)"
    },
    {
        "id": 482,
        "code": "12080",
        "METS": 11.5,
        "activityGroupId": 12,
        "description": "running, 7.5 mph (8 min/mile)"
    },
    {
        "id": 483,
        "code": "12090",
        "METS": 11.8,
        "activityGroupId": 12,
        "description": "running, 8 mph (7.5 min/mile)"
    },
    {
        "id": 484,
        "code": "12100",
        "METS": 12.3,
        "activityGroupId": 12,
        "description": "running, 8.6 mph (7 min/mile)"
    },
    {
        "id": 485,
        "code": "12110",
        "METS": 12.8,
        "activityGroupId": 12,
        "description": "running, 9 mph (6.5 min/mile)"
    },
    {
        "id": 486,
        "code": "12120",
        "METS": 14.5,
        "activityGroupId": 12,
        "description": "running, 10 mph (6 min/mile)"
    },
    {
        "id": 487,
        "code": "12130",
        "METS": 16,
        "activityGroupId": 12,
        "description": "running, 11 mph (5.5 min/mile)"
    },
    {
        "id": 488,
        "code": "12132",
        "METS": 19,
        "activityGroupId": 12,
        "description": "running, 12 mph (5 min/mile)"
    },
    {
        "id": 489,
        "code": "12134",
        "METS": 19.8,
        "activityGroupId": 12,
        "description": "running, 13 mph (4.6 min/mile)"
    },
    {
        "id": 490,
        "code": "12135",
        "METS": 23,
        "activityGroupId": 12,
        "description": "running, 14 mph (4.3 min/mile)"
    },
    {
        "id": 491,
        "code": "12140",
        "METS": 9,
        "activityGroupId": 12,
        "description": "running, cross country"
    },
    {
        "id": 492,
        "code": "12150",
        "METS": 8,
        "activityGroupId": 12,
        "description": "running, (Taylor code 200)"
    },
    {
        "id": 493,
        "code": "12170",
        "METS": 15,
        "activityGroupId": 12,
        "description": "running, stairs, up"
    },
    {
        "id": 494,
        "code": "12180",
        "METS": 10,
        "activityGroupId": 12,
        "description": "running, on a track, team practice"
    },
    {
        "id": 495,
        "code": "12190",
        "METS": 8,
        "activityGroupId": 12,
        "description": "running, training, pushing a wheelchair or baby carrier"
    },
    {
        "id": 496,
        "code": "12200",
        "METS": 13.3,
        "activityGroupId": 12,
        "description": "running, marathon"
    },
    {
        "id": 497,
        "code": "13000",
        "METS": 2.3,
        "activityGroupId": 13,
        "description": "getting ready for bed, general, standing"
    },
    {
        "id": 498,
        "code": "13009",
        "METS": 1.8,
        "activityGroupId": 13,
        "description": "sitting on toilet, eliminating while standing or "
                       "squating"
    },
    {
        "id": 499,
        "code": "13010",
        "METS": 1.5,
        "activityGroupId": 13,
        "description": "bathing, sitting"
    },
    {
        "id": 500,
        "code": "13020",
        "METS": 2.5,
        "activityGroupId": 13,
        "description": "dressing, undressing, standing or sitting"
    },
    {
        "id": 501,
        "code": "13030",
        "METS": 1.5,
        "activityGroupId": 13,
        "description": "eating, sitting"
    },
    {
        "id": 502,
        "code": "13035",
        "METS": 2,
        "activityGroupId": 13,
        "description": "talking and eating or eating only, standing"
    },
    {
        "id": 503,
        "code": "13036",
        "METS": 1.5,
        "activityGroupId": 13,
        "description": "taking medication, sitting or standing"
    },
    {
        "id": 504,
        "code": "13040",
        "METS": 2,
        "activityGroupId": 13,
        "description": "grooming, washing hands, shaving, brushing teeth, "
                       "putting on make-up, sitting or standing"
    },
    {
        "id": 505,
        "code": "13045",
        "METS": 2.5,
        "activityGroupId": 13,
        "description": "hairstyling, standing"
    },
    {
        "id": 506,
        "code": "13046",
        "METS": 1.3,
        "activityGroupId": 13,
        "description": "having hair or nails done by someone else, sitting"
    },
    {
        "id": 507,
        "code": "13050",
        "METS": 2,
        "activityGroupId": 13,
        "description": "showering, toweling off, standing"
    },
    {
        "id": 508,
        "code": "14010",
        "METS": 2.8,
        "activityGroupId": 14,
        "description": "active, vigorous effort"
    },
    {
        "id": 509,
        "code": "14020",
        "METS": 1.8,
        "activityGroupId": 14,
        "description": "general, moderate effort"
    },
    {
        "id": 510,
        "code": "14030",
        "METS": 1.3,
        "activityGroupId": 14,
        "description": "passive, light effort, kissing, hugging"
    },
    {
        "id": 511,
        "code": "15000",
        "METS": 5.5,
        "activityGroupId": 15,
        "description": "Alaska Native Games, Eskimo Olympics, general"
    },
    {
        "id": 512,
        "code": "15010",
        "METS": 4.3,
        "activityGroupId": 15,
        "description": "archery, non-hunting"
    },
    {
        "id": 513,
        "code": "15020",
        "METS": 7,
        "activityGroupId": 15,
        "description": "badminton, competitive (Taylor code 450)"
    },
    {
        "id": 514,
        "code": "15030",
        "METS": 5.5,
        "activityGroupId": 15,
        "description": "badminton, social singles and doubles, general"
    },
    {
        "id": 515,
        "code": "15040",
        "METS": 8,
        "activityGroupId": 15,
        "description": "basketball, game (Taylor code 490)"
    },
    {
        "id": 516,
        "code": "15050",
        "METS": 6,
        "activityGroupId": 15,
        "description": "basketball, non-game, general (Taylor code 480)"
    },
    {
        "id": 517,
        "code": "15055",
        "METS": 6.5,
        "activityGroupId": 15,
        "description": "basketball, general"
    },
    {
        "id": 518,
        "code": "15060",
        "METS": 7,
        "activityGroupId": 15,
        "description": "basketball, officiating (Taylor code 500)"
    },
    {
        "id": 519,
        "code": "15070",
        "METS": 4.5,
        "activityGroupId": 15,
        "description": "basketball, shooting baskets"
    },
    {
        "id": 520,
        "code": "15072",
        "METS": 9.3,
        "activityGroupId": 15,
        "description": "basketball, drills, practice"
    },
    {
        "id": 521,
        "code": "15075",
        "METS": 7.8,
        "activityGroupId": 15,
        "description": "basketball, wheelchair"
    },
    {
        "id": 522,
        "code": "15080",
        "METS": 2.5,
        "activityGroupId": 15,
        "description": "billiards"
    },
    {
        "id": 523,
        "code": "15090",
        "METS": 3,
        "activityGroupId": 15,
        "description": "bowling (Taylor code 390)"
    },
    {
        "id": 524,
        "code": "15092",
        "METS": 3.8,
        "activityGroupId": 15,
        "description": "bowling, indoor, bowling alley"
    },
    {
        "id": 525,
        "code": "15100",
        "METS": 12.8,
        "activityGroupId": 15,
        "description": "boxing, in ring, general"
    },
    {
        "id": 526,
        "code": "15110",
        "METS": 5.5,
        "activityGroupId": 15,
        "description": "boxing, punching bag"
    },
    {
        "id": 527,
        "code": "15120",
        "METS": 7.8,
        "activityGroupId": 15,
        "description": "boxing, sparring"
    },
    {
        "id": 528,
        "code": "15130",
        "METS": 7,
        "activityGroupId": 15,
        "description": "broomball"
    },
    {
        "id": 529,
        "code": "15135",
        "METS": 5.8,
        "activityGroupId": 15,
        "description": "children’s games, adults playing (e.g., hopscotch, "
                       "4-square, dodgeball, playground apparatus, t-ball, "
                       "tetherball, marbles, arcade games), moderate effort"
    },
    {
        "id": 530,
        "code": "15138",
        "METS": 6,
        "activityGroupId": 15,
        "description": "cheerleading, gymnastic moves, competitive"
    },
    {
        "id": 531,
        "code": "15140",
        "METS": 4,
        "activityGroupId": 15,
        "description": "coaching, football, soccer, basketball, baseball, "
                       "swimming, etc."
    },
    {
        "id": 532,
        "code": "15142",
        "METS": 8,
        "activityGroupId": 15,
        "description": "coaching, actively playing sport with players"
    },
    {
        "id": 533,
        "code": "15150",
        "METS": 4.8,
        "activityGroupId": 15,
        "description": "cricket, batting, bowling, fielding"
    },
    {
        "id": 534,
        "code": "15160",
        "METS": 3.3,
        "activityGroupId": 15,
        "description": "croquet"
    },
    {
        "id": 535,
        "code": "15170",
        "METS": 4,
        "activityGroupId": 15,
        "description": "curling"
    },
    {
        "id": 536,
        "code": "15180",
        "METS": 2.5,
        "activityGroupId": 15,
        "description": "darts, wall or lawn"
    },
    {
        "id": 537,
        "code": "15190",
        "METS": 6,
        "activityGroupId": 15,
        "description": "drag racing, pushing or driving a car"
    },
    {
        "id": 538,
        "code": "15192",
        "METS": 8.5,
        "activityGroupId": 15,
        "description": "auto racing, open wheel"
    },
    {
        "id": 539,
        "code": "15200",
        "METS": 6,
        "activityGroupId": 15,
        "description": "fencing"
    },
    {
        "id": 540,
        "code": "15210",
        "METS": 8,
        "activityGroupId": 15,
        "description": "football, competitive"
    },
    {
        "id": 541,
        "code": "15230",
        "METS": 8,
        "activityGroupId": 15,
        "description": "football, touch, flag, general (Taylor code 510)"
    },
    {
        "id": 542,
        "code": "15232",
        "METS": 4,
        "activityGroupId": 15,
        "description": "football, touch, flag, light effort"
    },
    {
        "id": 543,
        "code": "15235",
        "METS": 2.5,
        "activityGroupId": 15,
        "description": "football or baseball, playing catch"
    },
    {
        "id": 544,
        "code": "15240",
        "METS": 3,
        "activityGroupId": 15,
        "description": "frisbee playing, general"
    },
    {
        "id": 545,
        "code": "15250",
        "METS": 8,
        "activityGroupId": 15,
        "description": "frisbee, ultimate"
    },
    {
        "id": 546,
        "code": "15255",
        "METS": 4.8,
        "activityGroupId": 15,
        "description": "golf, general"
    },
    {
        "id": 547,
        "code": "15265",
        "METS": 4.3,
        "activityGroupId": 15,
        "description": "golf, walking, carrying clubs"
    },
    {
        "id": 548,
        "code": "15270",
        "METS": 3,
        "activityGroupId": 15,
        "description": "golf, miniature, driving range"
    },
    {
        "id": 549,
        "code": "15285",
        "METS": 5.3,
        "activityGroupId": 15,
        "description": "golf, walking, pulling clubs"
    },
    {
        "id": 550,
        "code": "15290",
        "METS": 3.5,
        "activityGroupId": 15,
        "description": "golf, using power cart (Taylor code 070)"
    },
    {
        "id": 551,
        "code": "15300",
        "METS": 3.8,
        "activityGroupId": 15,
        "description": "gymnastics, general"
    },
    {
        "id": 552,
        "code": "15310",
        "METS": 4,
        "activityGroupId": 15,
        "description": "hacky sack"
    },
    {
        "id": 553,
        "code": "15320",
        "METS": 12,
        "activityGroupId": 15,
        "description": "handball, general (Taylor code 520)"
    },
    {
        "id": 554,
        "code": "15330",
        "METS": 8,
        "activityGroupId": 15,
        "description": "handball, team"
    },
    {
        "id": 555,
        "code": "15335",
        "METS": 4,
        "activityGroupId": 15,
        "description": "high ropes course, multiple elements"
    },
    {
        "id": 556,
        "code": "15340",
        "METS": 3.5,
        "activityGroupId": 15,
        "description": "hang gliding"
    },
    {
        "id": 557,
        "code": "15350",
        "METS": 7.8,
        "activityGroupId": 15,
        "description": "hockey, field"
    },
    {
        "id": 558,
        "code": "15360",
        "METS": 8,
        "activityGroupId": 15,
        "description": "hockey, ice, general"
    },
    {
        "id": 559,
        "code": "15362",
        "METS": 10,
        "activityGroupId": 15,
        "description": "hockey, ice, competitive"
    },
    {
        "id": 560,
        "code": "15370",
        "METS": 5.5,
        "activityGroupId": 15,
        "description": "horseback riding, general"
    },
    {
        "id": 561,
        "code": "15375",
        "METS": 4.3,
        "activityGroupId": 15,
        "description": "horse chores, feeding, watering, cleaning stalls, "
                       "implied walking and lifting loads"
    },
    {
        "id": 562,
        "code": "15380",
        "METS": 4.5,
        "activityGroupId": 15,
        "description": "saddling, cleaning, grooming, harnessing and "
                       "unharnessing horse"
    },
    {
        "id": 563,
        "code": "15390",
        "METS": 5.8,
        "activityGroupId": 15,
        "description": "horseback riding, trotting"
    },
    {
        "id": 564,
        "code": "15395",
        "METS": 7.3,
        "activityGroupId": 15,
        "description": "horseback riding, canter or gallop"
    },
    {
        "id": 565,
        "code": "15400",
        "METS": 3.8,
        "activityGroupId": 15,
        "description": "horseback riding,walking"
    },
    {
        "id": 566,
        "code": "15402",
        "METS": 9,
        "activityGroupId": 15,
        "description": "horseback riding, jumping"
    },
    {
        "id": 567,
        "code": "15408",
        "METS": 1.8,
        "activityGroupId": 15,
        "description": "horse cart, driving, standing or sitting"
    },
    {
        "id": 568,
        "code": "15410",
        "METS": 3,
        "activityGroupId": 15,
        "description": "horseshoe pitching, quoits"
    },
    {
        "id": 569,
        "code": "15420",
        "METS": 12,
        "activityGroupId": 15,
        "description": "jai alai"
    },
    {
        "id": 570,
        "code": "15425",
        "METS": 5.3,
        "activityGroupId": 15,
        "description": "martial arts, different types, slower pace, novice "
                       "performers, practice"
    },
    {
        "id": 571,
        "code": "15430",
        "METS": 10.3,
        "activityGroupId": 15,
        "description": "martial arts, different types, moderate pace (e.g., "
                       "judo, jujitsu, karate, kick boxing, tae kwan do, "
                       "tai-bo, Muay Thai boxing)"
    },
    {
        "id": 572,
        "code": "15440",
        "METS": 4,
        "activityGroupId": 15,
        "description": "juggling"
    },
    {
        "id": 573,
        "code": "15450",
        "METS": 7,
        "activityGroupId": 15,
        "description": "kickball"
    },
    {
        "id": 574,
        "code": "15460",
        "METS": 8,
        "activityGroupId": 15,
        "description": "lacrosse"
    },
    {
        "id": 575,
        "code": "15465",
        "METS": 3.3,
        "activityGroupId": 15,
        "description": "lawn bowling, bocce ball, outdoor"
    },
    {
        "id": 576,
        "code": "15470",
        "METS": 4,
        "activityGroupId": 15,
        "description": "moto-cross, off-road motor sports, all-terrain "
                       "vehicle, general"
    },
    {
        "id": 577,
        "code": "15480",
        "METS": 9,
        "activityGroupId": 15,
        "description": "orienteering"
    },
    {
        "id": 578,
        "code": "15490",
        "METS": 10,
        "activityGroupId": 15,
        "description": "paddleball, competitive"
    },
    {
        "id": 579,
        "code": "15500",
        "METS": 6,
        "activityGroupId": 15,
        "description": "paddleball, casual, general (Taylor code 460)"
    },
    {
        "id": 580,
        "code": "15510",
        "METS": 8,
        "activityGroupId": 15,
        "description": "polo, on horseback"
    },
    {
        "id": 581,
        "code": "15520",
        "METS": 10,
        "activityGroupId": 15,
        "description": "racquetball, competitive"
    },
    {
        "id": 582,
        "code": "15530",
        "METS": 7,
        "activityGroupId": 15,
        "description": "racquetball, general (Taylor code 470)"
    },
    {
        "id": 583,
        "code": "15533",
        "METS": 8,
        "activityGroupId": 15,
        "description": "rock or mountain climbing (Taylor code 470) (Formerly"
                       " code = 17120)"
    },
    {
        "id": 584,
        "code": "15535",
        "METS": 7.5,
        "activityGroupId": 15,
        "description": "rock climbing, ascending rock, high difficulty"
    },
    {
        "id": 585,
        "code": "15537",
        "METS": 5.8,
        "activityGroupId": 15,
        "description": "rock climbing, ascending or traversing rock, "
                       "low-to-moderate difficulty"
    },
    {
        "id": 586,
        "code": "15540",
        "METS": 5,
        "activityGroupId": 15,
        "description": "rock climbing, rappelling"
    },
    {
        "id": 587,
        "code": "15542",
        "METS": 4,
        "activityGroupId": 15,
        "description": "rodeo sports, general, light effort"
    },
    {
        "id": 588,
        "code": "15544",
        "METS": 5.5,
        "activityGroupId": 15,
        "description": "rodeo sports, general, moderate effort"
    },
    {
        "id": 589,
        "code": "15546",
        "METS": 7,
        "activityGroupId": 15,
        "description": "rodeo sports, general, vigorous effort"
    },
    {
        "id": 590,
        "code": "15550",
        "METS": 12.3,
        "activityGroupId": 15,
        "description": "rope jumping, fast pace, 120-160 skips/min"
    },
    {
        "id": 591,
        "code": "15551",
        "METS": 11.8,
        "activityGroupId": 15,
        "description": "rope jumping, moderate pace, 100-120 skips/min, "
                       "general, 2 foot skip, plain bounce"
    },
    {
        "id": 592,
        "code": "15552",
        "METS": 8.8,
        "activityGroupId": 15,
        "description": "rope jumping, slow pace, < 100 skips/min, 2 foot "
                       "skip, rhythm bounce"
    },
    {
        "id": 593,
        "code": "15560",
        "METS": 8.3,
        "activityGroupId": 15,
        "description": "rugby, union, team, competitive"
    },
    {
        "id": 594,
        "code": "15562",
        "METS": 6.3,
        "activityGroupId": 15,
        "description": "rugby, touch, non-competitive"
    },
    {
        "id": 595,
        "code": "15570",
        "METS": 3,
        "activityGroupId": 15,
        "description": "shuffleboard"
    },
    {
        "id": 596,
        "code": "15580",
        "METS": 5,
        "activityGroupId": 15,
        "description": "skateboarding, general, moderate effort"
    },
    {
        "id": 597,
        "code": "15582",
        "METS": 6,
        "activityGroupId": 15,
        "description": "skateboarding, competitive, vigorous effort"
    },
    {
        "id": 598,
        "code": "15590",
        "METS": 7,
        "activityGroupId": 15,
        "description": "skating, roller (Taylor code 360)"
    },
    {
        "id": 599,
        "code": "15591",
        "METS": 7.5,
        "activityGroupId": 15,
        "description": "rollerblading, in-line skating, 14.4 km/h (9.0 mph), "
                       "recreational pace"
    },
    {
        "id": 600,
        "code": "15592",
        "METS": 9.8,
        "activityGroupId": 15,
        "description": "rollerblading, in-line skating, 17.7 km/h (11.0 mph), "
                       "moderate pace, exercise training"
    },
    {
        "id": 601,
        "code": "15593",
        "METS": 12.3,
        "activityGroupId": 15,
        "description": "rollerblading, in-line skating, 21.0 to 21.7 km/h ("
                       "13.0 to 13.6 mph), fast pace, exercise training"
    },
    {
        "id": 602,
        "code": "15594",
        "METS": 14,
        "activityGroupId": 15,
        "description": "rollerblading, in-line skating, 24.0 km/h (15.0 mph), "
                       "maximal effort"
    },
    {
        "id": 603,
        "code": "15600",
        "METS": 3.5,
        "activityGroupId": 15,
        "description": "skydiving, base jumping, bungee jumping"
    },
    {
        "id": 604,
        "code": "15605",
        "METS": 10,
        "activityGroupId": 15,
        "description": "soccer, competitive"
    },
    {
        "id": 605,
        "code": "15610",
        "METS": 7,
        "activityGroupId": 15,
        "description": "soccer, casual, general (Taylor code 540)"
    },
    {
        "id": 606,
        "code": "15620",
        "METS": 5,
        "activityGroupId": 15,
        "description": "softball or baseball, fast or slow pitch, general ("
                       "Taylor code 440)"
    },
    {
        "id": 607,
        "code": "15625",
        "METS": 4,
        "activityGroupId": 15,
        "description": "softball, practice"
    },
    {
        "id": 608,
        "code": "15630",
        "METS": 4,
        "activityGroupId": 15,
        "description": "softball, officiating"
    },
    {
        "id": 609,
        "code": "15640",
        "METS": 6,
        "activityGroupId": 15,
        "description": "softball,pitching"
    },
    {
        "id": 610,
        "code": "15645",
        "METS": 3.3,
        "activityGroupId": 15,
        "description": "sports spectator, very excited, emotional, physically"
                       " moving"
    },
    {
        "id": 611,
        "code": "15650",
        "METS": 12,
        "activityGroupId": 15,
        "description": "squash (Taylor code 530)"
    },
    {
        "id": 612,
        "code": "15652",
        "METS": 7.3,
        "activityGroupId": 15,
        "description": "squash, general"
    },
    {
        "id": 613,
        "code": "15660",
        "METS": 4,
        "activityGroupId": 15,
        "description": "table tennis, ping pong (Taylor code 410)"
    },
    {
        "id": 614,
        "code": "15670",
        "METS": 3,
        "activityGroupId": 15,
        "description": "tai chi, qi gong, general"
    },
    {
        "id": 615,
        "code": "15672",
        "METS": 1.5,
        "activityGroupId": 15,
        "description": "tai chi, qi gong, sitting, light effort"
    },
    {
        "id": 616,
        "code": "15675",
        "METS": 7.3,
        "activityGroupId": 15,
        "description": "tennis, general"
    },
    {
        "id": 617,
        "code": "15680",
        "METS": 6,
        "activityGroupId": 15,
        "description": "tennis, doubles (Taylor code 430)"
    },
    {
        "id": 618,
        "code": "15685",
        "METS": 4.5,
        "activityGroupId": 15,
        "description": "tennis, doubles"
    },
    {
        "id": 619,
        "code": "15690",
        "METS": 8,
        "activityGroupId": 15,
        "description": "tennis, singles (Taylor code 420)"
    },
    {
        "id": 620,
        "code": "15695",
        "METS": 5,
        "activityGroupId": 15,
        "description": "tennis, hitting balls, non-game play, moderate effort"
    },
    {
        "id": 621,
        "code": "15700",
        "METS": 3.5,
        "activityGroupId": 15,
        "description": "trampoline, recreational"
    },
    {
        "id": 622,
        "code": "15702",
        "METS": 4.5,
        "activityGroupId": 15,
        "description": "trampoline, competitive"
    },
    {
        "id": 623,
        "code": "15710",
        "METS": 4,
        "activityGroupId": 15,
        "description": "volleyball (Taylor code 400)"
    },
    {
        "id": 624,
        "code": "15711",
        "METS": 6,
        "activityGroupId": 15,
        "description": "volleyball, competitive, in gymnasium"
    },
    {
        "id": 625,
        "code": "15720",
        "METS": 3,
        "activityGroupId": 15,
        "description": "volleyball, non-competitive, 6 - 9 member team, general"
    },
    {
        "id": 626,
        "code": "15725",
        "METS": 8,
        "activityGroupId": 15,
        "description": "volleyball, beach, in sand"
    },
    {
        "id": 627,
        "code": "15730",
        "METS": 6,
        "activityGroupId": 15,
        "description": "wrestling (one match = 5 minutes)"
    },
    {
        "id": 628,
        "code": "15731",
        "METS": 7,
        "activityGroupId": 15,
        "description": "wallyball, general"
    },
    {
        "id": 629,
        "code": "15732",
        "METS": 4,
        "activityGroupId": 15,
        "description": "track and field (e.g., shot, discus, hammer throw)"
    },
    {
        "id": 630,
        "code": "15733",
        "METS": 6,
        "activityGroupId": 15,
        "description": "track and field (e.g., high jump, long jump, triple "
                       "jump, javelin, pole vault)"
    },
    {
        "id": 631,
        "code": "15734",
        "METS": 10,
        "activityGroupId": 15,
        "description": "track and field (e.g., steeplechase, hurdles)"
    },
    {
        "id": 632,
        "code": "16010",
        "METS": 2.5,
        "activityGroupId": 16,
        "description": "automobile or light truck (not a semi) driving"
    },
    {
        "id": 633,
        "code": "16015",
        "METS": 1.3,
        "activityGroupId": 16,
        "description": "riding in a car or truck"
    },
    {
        "id": 634,
        "code": "16016",
        "METS": 1.3,
        "activityGroupId": 16,
        "description": "riding in a bus or train"
    },
    {
        "id": 635,
        "code": "16020",
        "METS": 1.8,
        "activityGroupId": 16,
        "description": "flying airplane or helicopter"
    },
    {
        "id": 636,
        "code": "16030",
        "METS": 3.5,
        "activityGroupId": 16,
        "description": "motor scooter, motorcycle"
    },
    {
        "id": 637,
        "code": "16035",
        "METS": 6.3,
        "activityGroupId": 16,
        "description": "pulling rickshaw"
    },
    {
        "id": 638,
        "code": "16040",
        "METS": 6,
        "activityGroupId": 16,
        "description": "pushing plane in and out of hangar"
    },
    {
        "id": 639,
        "code": "16050",
        "METS": 2.5,
        "activityGroupId": 16,
        "description": "truck, semi, tractor, > 1 ton, or bus, driving"
    },
    {
        "id": 640,
        "code": "16060",
        "METS": 3.5,
        "activityGroupId": 16,
        "description": "walking for transportation, 2.8-3.2 mph, level, "
                       "moderate pace, firm surface"
    },
    {
        "id": 641,
        "code": "17010",
        "METS": 7,
        "activityGroupId": 17,
        "description": "backpacking (Taylor code 050)"
    },
    {
        "id": 642,
        "code": "17012",
        "METS": 7.8,
        "activityGroupId": 17,
        "description": "backpacking, hiking or organized walking with a daypack"
    },
    {
        "id": 643,
        "code": "17020",
        "METS": 5,
        "activityGroupId": 17,
        "description": "carrying 15 pound load (e.g. suitcase), level ground "
                       "or downstairs"
    },
    {
        "id": 644,
        "code": "17021",
        "METS": 2.3,
        "activityGroupId": 17,
        "description": "carrying 15 lb child, slow walking"
    },
    {
        "id": 645,
        "code": "17025",
        "METS": 8.3,
        "activityGroupId": 17,
        "description": "carrying load upstairs, general"
    },
    {
        "id": 646,
        "code": "17026",
        "METS": 5,
        "activityGroupId": 17,
        "description": "carrying 1 to 15 lb load, upstairs"
    },
    {
        "id": 647,
        "code": "17027",
        "METS": 6,
        "activityGroupId": 17,
        "description": "carrying 16 to 24 lb load, upstairs"
    },
    {
        "id": 648,
        "code": "17028",
        "METS": 8,
        "activityGroupId": 17,
        "description": "carrying 25 to 49 lb load, upstairs"
    },
    {
        "id": 649,
        "code": "17029",
        "METS": 10,
        "activityGroupId": 17,
        "description": "carrying 50 to 74 lb load, upstairs"
    },
    {
        "id": 650,
        "code": "17030",
        "METS": 12,
        "activityGroupId": 17,
        "description": "carrying > 74 lb load, upstairs"
    },
    {
        "id": 651,
        "code": "17031",
        "METS": 3.5,
        "activityGroupId": 17,
        "description": "loading /unloading a car, implied walking"
    },
    {
        "id": 652,
        "code": "17033",
        "METS": 6.3,
        "activityGroupId": 17,
        "description": "climbing hills, no load"
    },
    {
        "id": 653,
        "code": "17035",
        "METS": 6.5,
        "activityGroupId": 17,
        "description": "climbing hills with 0 to 9 lb load"
    },
    {
        "id": 654,
        "code": "17040",
        "METS": 7.3,
        "activityGroupId": 17,
        "description": "climbing hills with 10 to 20 lb load"
    },
    {
        "id": 655,
        "code": "17050",
        "METS": 8.3,
        "activityGroupId": 17,
        "description": "climbing hills with 21 to 42 lb load"
    },
    {
        "id": 656,
        "code": "17060",
        "METS": 9,
        "activityGroupId": 17,
        "description": "climbing hills with 42+ lb load"
    },
    {
        "id": 657,
        "code": "17070",
        "METS": 3.5,
        "activityGroupId": 17,
        "description": "descending stairs"
    },
    {
        "id": 658,
        "code": "17080",
        "METS": 6,
        "activityGroupId": 17,
        "description": "hiking, cross country (Taylor code 040)"
    },
    {
        "id": 659,
        "code": "17082",
        "METS": 5.3,
        "activityGroupId": 17,
        "description": "hiking or walking at a normal pace through fields and"
                       " hillsides"
    },
    {
        "id": 660,
        "code": "17085",
        "METS": 2.5,
        "activityGroupId": 17,
        "description": "bird watching, slow walk"
    },
    {
        "id": 661,
        "code": "17088",
        "METS": 4.5,
        "activityGroupId": 17,
        "description": "marching, moderate speed, military, no pack"
    },
    {
        "id": 662,
        "code": "17090",
        "METS": 8,
        "activityGroupId": 17,
        "description": "marching rapidly, military, no pack"
    },
    {
        "id": 663,
        "code": "17100",
        "METS": 4,
        "activityGroupId": 17,
        "description": "pushing or pulling stroller with child or walking "
                       "with children, 2.5 to 3.1 mph"
    },
    {
        "id": 664,
        "code": "17105",
        "METS": 3.8,
        "activityGroupId": 17,
        "description": "pushing a wheelchair, non-occupational"
    },
    {
        "id": 665,
        "code": "17110",
        "METS": 6.5,
        "activityGroupId": 17,
        "description": "race walking"
    },
    {
        "id": 666,
        "code": "17130",
        "METS": 8,
        "activityGroupId": 17,
        "description": "stair climbing, using or climbing up ladder (Taylor "
                       "code 030)"
    },
    {
        "id": 667,
        "code": "17133",
        "METS": 4,
        "activityGroupId": 17,
        "description": "stair climbing, slow pace"
    },
    {
        "id": 668,
        "code": "17134",
        "METS": 8.8,
        "activityGroupId": 17,
        "description": "stair climbing, fast pace"
    },
    {
        "id": 669,
        "code": "17140",
        "METS": 5,
        "activityGroupId": 17,
        "description": "using crutches"
    },
    {
        "id": 670,
        "code": "17150",
        "METS": 2,
        "activityGroupId": 17,
        "description": "walking, household"
    },
    {
        "id": 671,
        "code": "17151",
        "METS": 2,
        "activityGroupId": 17,
        "description": "walking, less than 2.0 mph, level, strolling, very slow"
    },
    {
        "id": 672,
        "code": "17152",
        "METS": 2.8,
        "activityGroupId": 17,
        "description": "walking, 2.0 mph, level, slow pace, firm surface"
    },
    {
        "id": 673,
        "code": "17160",
        "METS": 3.5,
        "activityGroupId": 17,
        "description": "walking for pleasure (Taylor code 010)"
    },
    {
        "id": 674,
        "code": "17161",
        "METS": 2.5,
        "activityGroupId": 17,
        "description": "walking from house to car or bus, from car or bus to "
                       "go places, from car or bus to and from the worksite"
    },
    {
        "id": 675,
        "code": "17162",
        "METS": 2.5,
        "activityGroupId": 17,
        "description": "walking to neighbor’s house or family’s house for "
                       "social reasons"
    },
    {
        "id": 676,
        "code": "17165",
        "METS": 3,
        "activityGroupId": 17,
        "description": "walking the dog"
    },
    {
        "id": 677,
        "code": "17170",
        "METS": 3,
        "activityGroupId": 17,
        "description": "walking, 2.5 mph, level, firm surface"
    },
    {
        "id": 678,
        "code": "17180",
        "METS": 3.3,
        "activityGroupId": 17,
        "description": "walking, 2.5 mph, downhill"
    },
    {
        "id": 679,
        "code": "17190",
        "METS": 3.5,
        "activityGroupId": 17,
        "description": "walking, 2.8 to 3.2 mph, level, moderate pace, "
                       "firm surface"
    },
    {
        "id": 680,
        "code": "17200",
        "METS": 4.3,
        "activityGroupId": 17,
        "description": "walking, 3.5 mph, level, brisk, firm surface, walking"
                       " for exercise"
    },
    {
        "id": 681,
        "code": "17210",
        "METS": 5.3,
        "activityGroupId": 17,
        "description": "walking, 2.9 to 3.5 mph, uphill, 1 to 5% grade"
    },
    {
        "id": 682,
        "code": "17211",
        "METS": 8,
        "activityGroupId": 17,
        "description": "walking, 2.9 to 3.5 mph, uphill, 6% to 15% grade"
    },
    {
        "id": 683,
        "code": "17220",
        "METS": 5,
        "activityGroupId": 17,
        "description": "walking, 4.0 mph, level, firm surface, very brisk pace"
    },
    {
        "id": 684,
        "code": "17230",
        "METS": 7,
        "activityGroupId": 17,
        "description": "walking, 4.5 mph, level, firm surface, very, very brisk"
    },
    {
        "id": 685,
        "code": "17231",
        "METS": 8.3,
        "activityGroupId": 17,
        "description": "walking, 5.0 mph, level, firm surface"
    },
    {
        "id": 686,
        "code": "17235",
        "METS": 9.8,
        "activityGroupId": 17,
        "description": "walking, 5.0 mph, uphill, 3% grade"
    },
    {
        "id": 687,
        "code": "17250",
        "METS": 3.5,
        "activityGroupId": 17,
        "description": "walking, for pleasure, work break"
    },
    {
        "id": 688,
        "code": "17260",
        "METS": 4.8,
        "activityGroupId": 17,
        "description": "walking, grass track"
    },
    {
        "id": 689,
        "code": "17262",
        "METS": 4.5,
        "activityGroupId": 17,
        "description": "walking, normal pace, plowed field or sand"
    },
    {
        "id": 690,
        "code": "17270",
        "METS": 4,
        "activityGroupId": 17,
        "description": "walking, to work or class (Taylor code 015)"
    },
    {
        "id": 691,
        "code": "17280",
        "METS": 2.5,
        "activityGroupId": 17,
        "description": "walking, to and from an outhouse"
    },
    {
        "id": 692,
        "code": "17302",
        "METS": 4.8,
        "activityGroupId": 17,
        "description": "walking, for exercise, 3.5 to 4 mph, with ski poles, "
                       "Nordic walking, level, moderate pace"
    },
    {
        "id": 693,
        "code": "17305",
        "METS": 9.5,
        "activityGroupId": 17,
        "description": "walking, for exercise, 5.0 mph, with ski poles, "
                       "Nordic walking, level, fast pace"
    },
    {
        "id": 694,
        "code": "17310",
        "METS": 6.8,
        "activityGroupId": 17,
        "description": "walking, for exercise, with ski poles, Nordic "
                       "walking, uphill"
    },
    {
        "id": 695,
        "code": "17320",
        "METS": 6,
        "activityGroupId": 17,
        "description": "walking, backwards, 3.5 mph, level"
    },
    {
        "id": 696,
        "code": "17325",
        "METS": 8,
        "activityGroupId": 17,
        "description": "walking, backwards, 3.5 mph, uphill, 5% grade"
    },
    {
        "id": 697,
        "code": "18010",
        "METS": 2.5,
        "activityGroupId": 18,
        "description": "boating, power, driving"
    },
    {
        "id": 698,
        "code": "18012",
        "METS": 1.3,
        "activityGroupId": 18,
        "description": "boating, power, passenger, light"
    },
    {
        "id": 699,
        "code": "18020",
        "METS": 4,
        "activityGroupId": 18,
        "description": "canoeing, on camping trip (Taylor code 270)"
    },
    {
        "id": 700,
        "code": "18025",
        "METS": 3.3,
        "activityGroupId": 18,
        "description": "canoeing, harvesting wild rice, knocking rice off the"
                       " stalks"
    },
    {
        "id": 701,
        "code": "18030",
        "METS": 7,
        "activityGroupId": 18,
        "description": "canoeing, portaging"
    },
    {
        "id": 702,
        "code": "18040",
        "METS": 2.8,
        "activityGroupId": 18,
        "description": "canoeing, rowing, 2.0-3.9 mph, light effort"
    },
    {
        "id": 703,
        "code": "18050",
        "METS": 5.8,
        "activityGroupId": 18,
        "description": "canoeing, rowing, 4.0-5.9 mph, moderate effort"
    },
    {
        "id": 704,
        "code": "18060",
        "METS": 12.5,
        "activityGroupId": 18,
        "description": "canoeing, rowing, kayaking, competition, >6 mph, "
                       "vigorous effort"
    },
    {
        "id": 705,
        "code": "18070",
        "METS": 3.5,
        "activityGroupId": 18,
        "description": "canoeing, rowing, for pleasure, general (Taylor code "
                       "250)"
    },
    {
        "id": 706,
        "code": "18080",
        "METS": 12,
        "activityGroupId": 18,
        "description": "canoeing, rowing, in competition, or crew or sculling "
                       "(Taylor code 260)"
    },
    {
        "id": 707,
        "code": "18090",
        "METS": 3,
        "activityGroupId": 18,
        "description": "diving, springboard or platform"
    },
    {
        "id": 708,
        "code": "18100",
        "METS": 5,
        "activityGroupId": 18,
        "description": "kayaking, moderate effort"
    },
    {
        "id": 709,
        "code": "18110",
        "METS": 4,
        "activityGroupId": 18,
        "description": "paddle boat"
    },
    {
        "id": 710,
        "code": "18120",
        "METS": 3,
        "activityGroupId": 18,
        "description": "sailing, boat and board sailing, windsurfing, "
                       "ice sailing, general (Taylor code 235)"
    },
    {
        "id": 711,
        "code": "18130",
        "METS": 4.5,
        "activityGroupId": 18,
        "description": "sailing, in competition"
    },
    {
        "id": 712,
        "code": "18140",
        "METS": 3.3,
        "activityGroupId": 18,
        "description": "sailing, Sunfish/Laser/Hobby Cat, Keel boats, ocean sailing, yachting, leisure"
    },
    {
        "id": 713,
        "code": "18150",
        "METS": 6,
        "activityGroupId": 18,
        "description": "skiing, water or wakeboarding (Taylor code 220)"
    },
    {
        "id": 714,
        "code": "18160",
        "METS": 7,
        "activityGroupId": 18,
        "description": "jet skiing, driving, in water"
    },
    {
        "id": 715,
        "code": "18180",
        "METS": 15.8,
        "activityGroupId": 18,
        "description": "skindiving, fast"
    },
    {
        "id": 716,
        "code": "18190",
        "METS": 11.8,
        "activityGroupId": 18,
        "description": "skindiving, moderate"
    },
    {
        "id": 717,
        "code": "18200",
        "METS": 7,
        "activityGroupId": 18,
        "description": "skindiving, scuba diving, general (Taylor code 310)"
    },
    {
        "id": 718,
        "code": "18210",
        "METS": 5,
        "activityGroupId": 18,
        "description": "snorkeling (Taylor code 310)"
    },
    {
        "id": 719,
        "code": "18220",
        "METS": 3,
        "activityGroupId": 18,
        "description": "surfing, body or board, general"
    },
    {
        "id": 720,
        "code": "18222",
        "METS": 5,
        "activityGroupId": 18,
        "description": "surfing, body or board, competitive"
    },
    {
        "id": 721,
        "code": "18225",
        "METS": 6,
        "activityGroupId": 18,
        "description": "paddle boarding, standing"
    },
    {
        "id": 722,
        "code": "18230",
        "METS": 9.8,
        "activityGroupId": 18,
        "description": "swimming laps, freestyle, fast, vigorous effort"
    },
    {
        "id": 723,
        "code": "18240",
        "METS": 5.8,
        "activityGroupId": 18,
        "description": "swimming laps, freestyle, front crawl, slow, light or moderate effort"
    },
    {
        "id": 724,
        "code": "18250",
        "METS": 9.5,
        "activityGroupId": 18,
        "description": "swimming, backstroke, general, training or competition"
    },
    {
        "id": 725,
        "code": "18255",
        "METS": 4.8,
        "activityGroupId": 18,
        "description": "swimming, backstroke, recreational"
    },
    {
        "id": 726,
        "code": "18260",
        "METS": 10.3,
        "activityGroupId": 18,
        "description": "swimming, breaststroke, general, training or competition"
    },
    {
        "id": 727,
        "code": "18265",
        "METS": 5.3,
        "activityGroupId": 18,
        "description": "swimming, breaststroke, recreational"
    },
    {
        "id": 728,
        "code": "18270",
        "METS": 13.8,
        "activityGroupId": 18,
        "description": "swimming, butterfly, general"
    },
    {
        "id": 729,
        "code": "18280",
        "METS": 10,
        "activityGroupId": 18,
        "description": "swimming, crawl, fast speed, ~75 yards/minute, vigorous effort"
    },
    {
        "id": 730,
        "code": "18290",
        "METS": 8.3,
        "activityGroupId": 18,
        "description": "swimming, crawl, medium speed, ~50 yards/minute, vigorous effort"
    },
    {
        "id": 731,
        "code": "18300",
        "METS": 6,
        "activityGroupId": 18,
        "description": "swimming, lake, ocean, river (Taylor codes 280, 295)"
    },
    {
        "id": 732,
        "code": "18310",
        "METS": 6,
        "activityGroupId": 18,
        "description": "swimming, leisurely, not lap swimming, general"
    },
    {
        "id": 733,
        "code": "18320",
        "METS": 7,
        "activityGroupId": 18,
        "description": "swimming, sidestroke, general"
    },
    {
        "id": 734,
        "code": "18330",
        "METS": 8,
        "activityGroupId": 18,
        "description": "swimming, synchronized"
    },
    {
        "id": 735,
        "code": "18340",
        "METS": 9.8,
        "activityGroupId": 18,
        "description": "swimming, treading water, fast, vigorous effort"
    },
    {
        "id": 736,
        "code": "18350",
        "METS": 3.5,
        "activityGroupId": 18,
        "description": "swimming, treading water, moderate effort, general"
    },
    {
        "id": 737,
        "code": "18352",
        "METS": 2.3,
        "activityGroupId": 18,
        "description": "tubing, floating on a river, general"
    },
    {
        "id": 738,
        "code": "18355",
        "METS": 5.5,
        "activityGroupId": 18,
        "description": "water aerobics, water calisthenics"
    },
    {
        "id": 739,
        "code": "18360",
        "METS": 10,
        "activityGroupId": 18,
        "description": "water polo"
    },
    {
        "id": 740,
        "code": "18365",
        "METS": 3,
        "activityGroupId": 18,
        "description": "water volleyball"
    },
    {
        "id": 741,
        "code": "18366",
        "METS": 9.8,
        "activityGroupId": 18,
        "description": "water jogging"
    },
    {
        "id": 742,
        "code": "18367",
        "METS": 2.5,
        "activityGroupId": 18,
        "description": "water walking, light effort, slow pace"
    },
    {
        "id": 743,
        "code": "18368",
        "METS": 4.5,
        "activityGroupId": 18,
        "description": "water walking, moderate effort, moderate pace"
    },
    {
        "id": 744,
        "code": "18369",
        "METS": 6.8,
        "activityGroupId": 18,
        "description": "water walking, vigorous effort, brisk pace"
    },
    {
        "id": 745,
        "code": "18370",
        "METS": 5,
        "activityGroupId": 18,
        "description": "whitewater rafting, kayaking, or canoeing"
    },
    {
        "id": 746,
        "code": "18380",
        "METS": 5,
        "activityGroupId": 18,
        "description": "windsurfing, not pumping for speed"
    },
    {
        "id": 747,
        "code": "18385",
        "METS": 11,
        "activityGroupId": 18,
        "description": "windsurfing or kitesurfing, crossing trial"
    },
    {
        "id": 748,
        "code": "18390",
        "METS": 13.5,
        "activityGroupId": 18,
        "description": "windsurfing, competition, pumping for speed"
    },
    {
        "id": 749,
        "code": "19005",
        "METS": 7.5,
        "activityGroupId": 19,
        "description": "dog sledding, mushing"
    },
    {
        "id": 750,
        "code": "19006",
        "METS": 2.5,
        "activityGroupId": 19,
        "description": "dog sledding, passenger"
    },
    {
        "id": 751,
        "code": "19010",
        "METS": 6,
        "activityGroupId": 19,
        "description": "moving ice house, set up/drill holes"
    },
    {
        "id": 752,
        "code": "19011",
        "METS": 2,
        "activityGroupId": 19,
        "description": "ice fishing, sitting"
    },
    {
        "id": 753,
        "code": "19018",
        "METS": 14,
        "activityGroupId": 19,
        "description": "skating, ice dancing"
    },
    {
        "id": 754,
        "code": "19020",
        "METS": 5.5,
        "activityGroupId": 19,
        "description": "skating, ice, 9 mph or less"
    },
    {
        "id": 755,
        "code": "19030",
        "METS": 7,
        "activityGroupId": 19,
        "description": "skating, ice, general (Taylor code 360)"
    },
    {
        "id": 756,
        "code": "19040",
        "METS": 9,
        "activityGroupId": 19,
        "description": "skating, ice, rapidly, more than 9 mph, not competitive"
    },
    {
        "id": 757,
        "code": "19050",
        "METS": 13.3,
        "activityGroupId": 19,
        "description": "skating, speed, competitive"
    },
    {
        "id": 758,
        "code": "19060",
        "METS": 7,
        "activityGroupId": 19,
        "description": "ski jumping, climb up carrying skis"
    },
    {
        "id": 759,
        "code": "19075",
        "METS": 7,
        "activityGroupId": 19,
        "description": "skiing, general"
    },
    {
        "id": 760,
        "code": "19080",
        "METS": 6.8,
        "activityGroupId": 19,
        "description": "skiing, cross country, 2.5 mph, slow or light effort, ski walking"
    },
    {
        "id": 761,
        "code": "19090",
        "METS": 9,
        "activityGroupId": 19,
        "description": "skiing, cross country, 4.0-4.9 mph, moderate speed and effort, general"
    },
    {
        "id": 762,
        "code": "19100",
        "METS": 12.5,
        "activityGroupId": 19,
        "description": "skiing, cross country, 5.0-7.9 mph, brisk speed, vigorous effort"
    },
    {
        "id": 763,
        "code": "19110",
        "METS": 15,
        "activityGroupId": 19,
        "description": "skiing, cross country, >8.0 mph, elite skier, racing"
    },
    {
        "id": 764,
        "code": "19130",
        "METS": 15.5,
        "activityGroupId": 19,
        "description": "skiing, cross country, hard snow, uphill, maximum, snow mountaineering"
    },
    {
        "id": 765,
        "code": "19135",
        "METS": 13.3,
        "activityGroupId": 19,
        "description": "skiing, cross-country, skating"
    },
    {
        "id": 766,
        "code": "19140",
        "METS": 13.5,
        "activityGroupId": 19,
        "description": "skiing, cross-country, biathlon, skating technique"
    },
    {
        "id": 767,
        "code": "19150",
        "METS": 4.3,
        "activityGroupId": 19,
        "description": "skiing, downhill, alpine or snowboarding, light effort, active time only"
    },
    {
        "id": 768,
        "code": "19160",
        "METS": 5.3,
        "activityGroupId": 19,
        "description": "skiing, downhill, alpine or snowboarding, moderate effort, general, active time only"
    },
    {
        "id": 769,
        "code": "19170",
        "METS": 8,
        "activityGroupId": 19,
        "description": "skiing, downhill, vigorous effort, racing"
    },
    {
        "id": 770,
        "code": "19175",
        "METS": 12.5,
        "activityGroupId": 19,
        "description": "skiing, roller, elite racers"
    },
    {
        "id": 771,
        "code": "19180",
        "METS": 7,
        "activityGroupId": 19,
        "description": "sledding, tobogganing, bobsledding, luge (Taylor code 370)"
    },
    {
        "id": 772,
        "code": "19190",
        "METS": 5.3,
        "activityGroupId": 19,
        "description": "snow shoeing, moderate effort"
    },
    {
        "id": 773,
        "code": "19192",
        "METS": 10,
        "activityGroupId": 19,
        "description": "snow shoeing, vigorous effort"
    },
    {
        "id": 774,
        "code": "19200",
        "METS": 3.5,
        "activityGroupId": 19,
        "description": "snowmobiling, driving, moderate"
    },
    {
        "id": 775,
        "code": "19202",
        "METS": 2,
        "activityGroupId": 19,
        "description": "snowmobiling, passenger"
    },
    {
        "id": 776,
        "code": "19252",
        "METS": 5.3,
        "activityGroupId": 19,
        "description": "snow shoveling, by hand, moderate effort"
    },
    {
        "id": 777,
        "code": "19254",
        "METS": 7.5,
        "activityGroupId": 19,
        "description": "snow shoveling, by hand, vigorous effort"
    },
    {
        "id": 778,
        "code": "19260",
        "METS": 2.5,
        "activityGroupId": 19,
        "description": "snow blower, walking and pushing"
    },
    {
        "id": 779,
        "code": "20000",
        "METS": 1.3,
        "activityGroupId": 20,
        "description": "sitting in church, in service, attending a ceremony, sitting quietly"
    },
    {
        "id": 780,
        "code": "20001",
        "METS": 2,
        "activityGroupId": 20,
        "description": "sitting, playing an instrument at church"
    },
    {
        "id": 781,
        "code": "20005",
        "METS": 1.8,
        "activityGroupId": 20,
        "description": "sitting in church, talking or singing, attending a ceremony, sitting, active participation"
    },
    {
        "id": 782,
        "code": "20010",
        "METS": 1.3,
        "activityGroupId": 20,
        "description": "sitting, reading religious materials at home"
    },
    {
        "id": 783,
        "code": "20015",
        "METS": 1.3,
        "activityGroupId": 20,
        "description": "standing quietly in church, attending a ceremony"
    },
    {
        "id": 784,
        "code": "20020",
        "METS": 2,
        "activityGroupId": 20,
        "description": "standing, singing in church, attending a ceremony, standing, active participation"
    },
    {
        "id": 785,
        "code": "20025",
        "METS": 1.3,
        "activityGroupId": 20,
        "description": "kneeling in church or at home, praying"
    },
    {
        "id": 786,
        "code": "20030",
        "METS": 1.8,
        "activityGroupId": 20,
        "description": "standing, talking in church"
    },
    {
        "id": 787,
        "code": "20035",
        "METS": 2,
        "activityGroupId": 20,
        "description": "walking in church"
    },
    {
        "id": 788,
        "code": "20036",
        "METS": 2,
        "activityGroupId": 20,
        "description": "walking, less than 2.0 mph, very slow"
    },
    {
        "id": 789,
        "code": "20037",
        "METS": 3.5,
        "activityGroupId": 20,
        "description": "walking, 3.0 mph, moderate speed, not carrying anything"
    },
    {
        "id": 790,
        "code": "20038",
        "METS": 4.3,
        "activityGroupId": 20,
        "description": "walking, 3.5 mph, brisk speed, not carrying anything"
    },
    {
        "id": 791,
        "code": "20039",
        "METS": 2,
        "activityGroupId": 20,
        "description": "walk/stand combination for religious purposes, usher"
    },
    {
        "id": 792,
        "code": "20040",
        "METS": 5,
        "activityGroupId": 20,
        "description": "praise with dance or run, spiritual dancing in church"
    },
    {
        "id": 793,
        "code": "20045",
        "METS": 2.5,
        "activityGroupId": 20,
        "description": "serving food at church"
    },
    {
        "id": 794,
        "code": "20046",
        "METS": 2,
        "activityGroupId": 20,
        "description": "preparing food at church"
    },
    {
        "id": 795,
        "code": "20047",
        "METS": 3.3,
        "activityGroupId": 20,
        "description": "washing dishes, cleaning kitchen at church"
    },
    {
        "id": 796,
        "code": "20050",
        "METS": 1.5,
        "activityGroupId": 20,
        "description": "eating at church"
    },
    {
        "id": 797,
        "code": "20055",
        "METS": 2,
        "activityGroupId": 20,
        "description": "eating/talking at church or standing eating, American Indian Feast days"
    },
    {
        "id": 798,
        "code": "20060",
        "METS": 3.3,
        "activityGroupId": 20,
        "description": "cleaning church"
    },
    {
        "id": 799,
        "code": "20061",
        "METS": 4,
        "activityGroupId": 20,
        "description": "general yard work at church"
    },
    {
        "id": 800,
        "code": "20065",
        "METS": 3.5,
        "activityGroupId": 20,
        "description": "standing, moderate effort (e.g., lifting heavy objects, assembling at fast rate)"
    },
    {
        "id": 801,
        "code": "20095",
        "METS": 4.5,
        "activityGroupId": 20,
        "description": "Standing, moderate-to-heavy effort, manual labor, lifting ? 50 lbs, heavy maintenance"
    },
    {
        "id": 802,
        "code": "20100",
        "METS": 1.3,
        "activityGroupId": 20,
        "description": "typing, electric, manual, or computer"
    },
    {
        "id": 803,
        "code": "21000",
        "METS": 1.5,
        "activityGroupId": 21,
        "description": "sitting, meeting, general, and/or with talking involved"
    },
    {
        "id": 804,
        "code": "21005",
        "METS": 1.5,
        "activityGroupId": 21,
        "description": "sitting, light office work, in general"
    },
    {
        "id": 805,
        "code": "21010",
        "METS": 2.5,
        "activityGroupId": 21,
        "description": "sitting, moderate work"
    },
    {
        "id": 806,
        "code": "21015",
        "METS": 2.3,
        "activityGroupId": 21,
        "description": "standing, light work (filing, talking, assembling)"
    },
    {
        "id": 807,
        "code": "21016",
        "METS": 2,
        "activityGroupId": 21,
        "description": "sitting, child care, only active periods"
    },
    {
        "id": 808,
        "code": "21017",
        "METS": 3,
        "activityGroupId": 21,
        "description": "standing, child care, only active periods"
    },
    {
        "id": 809,
        "code": "21018",
        "METS": 3.5,
        "activityGroupId": 21,
        "description": "walk/run play with children, moderate, only active periods"
    },
    {
        "id": 810,
        "code": "21019",
        "METS": 5.8,
        "activityGroupId": 21,
        "description": "walk/run play with children, vigorous, only active periods"
    },
    {
        "id": 811,
        "code": "21020",
        "METS": 3,
        "activityGroupId": 21,
        "description": "standing, light/moderate work (e.g., pack boxes, assemble/repair, set up chairs/furniture)"
    },
    {
        "id": 812,
        "code": "21025",
        "METS": 3.5,
        "activityGroupId": 21,
        "description": "standing, moderate (lifting 50 lbs., assembling at fast rate)"
    },
    {
        "id": 813,
        "code": "21030",
        "METS": 4.5,
        "activityGroupId": 21,
        "description": "standing, moderate/heavy work"
    },
    {
        "id": 814,
        "code": "21035",
        "METS": 1.3,
        "activityGroupId": 21,
        "description": "typing, electric, manual, or computer"
    },
    {
        "id": 815,
        "code": "21040",
        "METS": 2,
        "activityGroupId": 21,
        "description": "walking, less than 2.0 mph, very slow"
    },
    {
        "id": 816,
        "code": "21045",
        "METS": 3.5,
        "activityGroupId": 21,
        "description": "walking, 3.0 mph, moderate speed, not carrying anything"
    },
    {
        "id": 817,
        "code": "21050",
        "METS": 4.3,
        "activityGroupId": 21,
        "description": "walking, 3.5 mph, brisk speed, not carrying anything"
    },
    {
        "id": 818,
        "code": "21055",
        "METS": 3.5,
        "activityGroupId": 21,
        "description": "walking, 2.5 mph slowly and carrying objects less than 25 lbs"
    },
    {
        "id": 819,
        "code": "21060",
        "METS": 4.5,
        "activityGroupId": 21,
        "description": "walking, 3.0 mph moderately and carrying objects less than 25 lbs, pushing something"
    },
    {
        "id": 820,
        "code": "21065",
        "METS": 4.8,
        "activityGroupId": 21,
        "description": "walking, 3.5 mph, briskly and carrying objects less than 25 lbs"
    },
    {
        "id": 821,
        "code": "21070",
        "METS": 3,
        "activityGroupId": 21,
        "description": "walk/stand combination, for volunteer purposes"
    }
]



In [26]:
len(data)

821

In [29]:
"INSERT INTO PhysicalActivity (code, name, METS) \
              VALUES;"
insert_s = " ({}, \"{}\", {})"
statements = []
for i in data:
    statements.append(insert_s.format(i["code"], i["description"], i["METS"]))

print("INSERT INTO PhysicalActivity (code, name, METS) \
              VALUES {};".format(",\n".join(statements)))

INSERT INTO PhysicalActivity (code, name, METS)               VALUES  (01003, "bicycling, mountain, uphill, vigorous", 14),
 (01004, "bicycling, mountain, competitive, racing", 16),
 (01008, "bicycling, BMX", 8.5),
 (01009, "bicycling, mountain, general", 8.5),
 (01010, "bicycling, <10 mph, leisure, to work or for pleasure (Taylor code 115)", 4),
 (01011, "bicycling, to/from work, self selected pace", 6.8),
 (01013, "bicycling, on dirt or farm road, moderate pace", 5.8),
 (01015, "bicycling, general", 7.5),
 (01018, "bicycling, leisure, 5.5 mph", 3.5),
 (01019, "bicycling, leisure, 9.4 mph", 5.8),
 (01020, "bicycling, 10-11.9 mph, leisure, slow, light effort", 6.8),
 (01030, "bicycling, 12-13.9 mph, leisure, moderate effort", 8),
 (01040, "bicycling, 14-15.9 mph, racing or leisure, fast, vigorous effort", 10),
 (01050, "bicycling, 16-19 mph, racing/not drafting or > 19 mph drafting, very fast, racing general", 12),
 (01060, "bicycling, > 20 mph, racing, not drafting", 15.8),
 (01065, "